In [24]:
import pandas as pd
import numpy as np
import datetime
import argparse
import shutil
import sys
import os

from docplex.mp.model import Model

In [25]:
#CAMBIAR CUANDO SE QUIERA CORRER DESDE ACA O CUANDO SE QUIERA DEESCARGAR UNA NUEVA VERSION PARA CORRER AUTOMATICAMENTE

#ap = argparse.ArgumentParser()
#ap.add_argument("-m", "--manual", required=False, help="Indicar si es corrida manual corta (1) o larga (2)", default= 0)
#args = vars(ap.parse_args())

args = {'manual': '1'}

## Paths

In [26]:
# Definimos los paths a los cuales vamos a ir a buscar los excels necesarios
# y donde vamos a depositar el output.

path_excels = '../Archivos_Variables/'


path_fijos = '../fijos/'


carpeta_a_escuchar = 'C:/Users/TALIGENT/Documents/De Angelis Juan Bautista/Celula AA/1- Optimizador de despachos/malvina.bernal'

archivo_input = '../inputs.csv'

archivos = {'yva': None, 'yq10': None, 'embolse': None,'recepciones': None, 'yvatr': None}

## Funciones Útiles

In [27]:
# Esta función suma dias habiles a una fecha que le pase

def proximo_dia_habil(fecha, dias_a_sumar): 
    proximo_dia_habil = fecha
    while dias_a_sumar != 0:
        proximo_dia_habil += datetime.timedelta(days=1)
        if proximo_dia_habil.weekday() >= 5:
            continue
        dias_a_sumar -= 1       
    return proximo_dia_habil


# Esta función arma el nombre del output dependiendo del día actual y el numero de corrida

def armar_output():
        i = 0
        path_intermedio = '../Output/' + str(datetime.date.today())
        try:
            os.mkdir(path_intermedio)
                
        except FileExistsError:
                pass
            
        while True:
            path_out = path_intermedio + '/run_' + str(i)
            try:
                os.mkdir(path_out)
                break
            except FileExistsError:
                i += 1
                continue
        return path_out
    
    
# Esta función toma los archivos que se van a usar para el modelo y los copia en la carpeta output


def copiar_archivos(path_out):
        shutil.copy(path_excels + '/' + archivos['yva'], path_out + '/' + archivos['yva'])
        shutil.copy(path_excels + '/' + archivos['yq10'], path_out + '/' + archivos['yq10'])
        shutil.copy(path_fijos + '/' + archivos['embolse'], path_out + '/' + archivos['embolse'])
        shutil.copy(path_excels + '/' + archivos['yvatr'], path_out + '/' + archivos['yvatr'])
        inputs.to_csv(path_out + '/inputs.csv', sep=';', index= False)
        

# Esta función mira los archivos fijos y los identifica para leerlos posteriormente


def identificar_archivos_fijos(path, archivos):
    for file in os.listdir(path):
        try:
            split1 = file.split('_')[1]
            if split1 == 'EMBOLSE':
                    archivos['embolse'] = file
            elif split1 == 'RECEPCIONCENTROS':
                archivos['recepciones'] = file
            else: 
                pass
        except: pass
        
        
# Esta función mira los archivos variables y los identifica para posterior lectura

# CAMBIO ACÁ
def identificar_archivos(path, archivos):
    for file in os.listdir(path):
        split1 = file.split('_')[1]
        if split1 == 'YVADKLT':
            archivos['yva'] = file
        elif split1 == 'YQ10':
            archivos['yq10'] = file
        elif split1 == 'YVATRDKLT':
            archivos['yvatr'] = file
        else: 
            pass
        

# Funcion que se usa en el proceso de verificar que tenemos todos los archivos necesarios


def tengo_archivos(archivos):
    return all(archivos.values())


## Inputs

In [28]:
# BUSCO LOS ARCHIVOS INPUT

identificar_archivos(path_excels, archivos)

identificar_archivos_fijos(path_fijos, archivos)

if not(tengo_archivos(archivos)):
    print('Falta YVA o YQ10 o Embolse')
    sys.exit()

path_out = armar_output()

In [29]:
dateparse = lambda x: pd.datetime.strptime(str(x), '%d.%m.%Y')

In [30]:
# Lectura de la YVA
# CAMBIO ACÁ
data = pd.read_excel(path_excels + '/' + archivos['yva'], sheet_name= 'Sheet1')#, parse_dates=['Fecha_doc', 'Fe_SM_real','FePrefEnt_Cab', 'Fe_entrega','FechaFact'], date_parser=dateparse)

fecha_ultimo_pedido = data['Fe_entrega'].max()

In [31]:
# Si la corrida es manual se toman los inputs originales

if int(args['manual']) == 1:
    
    inputs = pd.read_csv(archivo_input, sep=';', encoding='mbcs')


# Corrida automatica a largo plazo

elif int(args['manual']) == 2:
    
    inputs = pd.read_csv(archivo_input, sep=';', encoding='mbcs')
    
    inputs['VALOR'].iloc[0] = datetime.date.today()
    
    inputs['VALOR'].iloc[1] = fecha_ultimo_pedido


# Si es automatica se inputa el dia actual y se toman 90 días hábiles en adelante

else:
    
    inputs = pd.read_csv(archivo_input, sep=';', encoding='mbcs')
    
    inputs['VALOR'].iloc[0] = datetime.date.today()
    
    inputs['VALOR'].iloc[1] = proximo_dia_habil(inputs['VALOR'].iloc[0], 90)

In [32]:
inputs

,VALOR,DESCRIPCION
0,10/2/2023,Día a partir del cual se quiere calcular (dia ...
1,7/7/2023,Día hasta el cual se quiere calcular (ultimo d...
2,NO,Se desea usar la fecha temprana de siembra ('...
3,NO,Se desea usar la fecha tardía de siembra ('SI...
4,12500,Capacidad logistica diaria (SSU)
5,3000,Capacidad REAP por dia (SSU)
6,Q,Ultima campaña despachable
7,SI,Calcular fechas erroneas ('SI' o 'NO')
8,NO,Correr Por OTD ('SI' o 'NO')


In [33]:
# Copio los archivos en la carpeta de outputs
copiar_archivos(path_out) 

In [34]:
# INPUTS QUE SE VAN A PODER CAMBIAR

fecha_de_hoy = pd.to_datetime(inputs['VALOR'].iloc[0], format='%d/%m/%Y') #Fecha del dia en que se hace la optimizacion

ultimo_dia = pd.to_datetime(inputs['VALOR'].iloc[1], format='%d/%m/%Y')

dias_habiles_disponibles = np.busday_count(fecha_de_hoy.date(), ultimo_dia.date()) # Dias habiles disponibles para despachar o hacer REAP

USAR_FECHA_SIEMBRA_TEMPRANA = True if inputs['VALOR'].iloc[2] == 'SI' else False

USAR_FECHA_SIEMBRA_TARDIA = True if inputs['VALOR'].iloc[3] == 'SI' else False

capacidad_despacho_por_dia = int(inputs['VALOR'].iloc[4]) #Cuantos SSU se pueden despachar por dia (segun camiones)

capacidad_reap_por_dia = int(inputs['VALOR'].iloc[5])

ultima_campaña_despachable = inputs['VALOR'].iloc[6]

calcular_fechas_erroneas = True if inputs['VALOR'].iloc[7] == 'SI' else False

nombre_output = path_out + '/OPTIMIZADODISTRIBUCION_' + fecha_de_hoy.strftime("%Y%m%d") + '_' + ultimo_dia.strftime("%Y%m%d") + '_ROJAS' + '.xlsx'

correr_otd = True if inputs['VALOR'].iloc[8] == 'SI' else False

In [35]:
# Lectura de todos los archivos necesarios

yq10 = pd.read_excel(path_excels + archivos['yq10']) #,parse_dates=['Fecha DE'], date_parser=dateparse)

embolse = pd.read_excel(path_fijos + archivos['embolse'], skiprows=5, sheet_name='Hoja 1')

prop = pd.read_csv(path_fijos + 'proporcion_de_material_por_hibrido')

yvatr = pd.read_excel(path_excels + archivos['yvatr'], sheet_name='Sheet1') #sheet_name='YVA TR GRAL')

prioridades = pd.read_excel(path_fijos + 'Prioridades Distribución FEFO.xlsx')

zonas = pd.read_excel(path_fijos + 'Detalle Distrib - Final.xlsx', skiprows=3)

recep = pd.read_excel(path_fijos + '/'  + archivos['recepciones'])

# CAMBIO ACÁ

# maestro = pd.read_excel('C:/FTPS/malvina.bernal/fijos/Grupo de clientes.xlsx', sheet_name= 'Base clientes')

## Embolse

In [36]:
embolse.drop(columns= ['Unnamed: 0', 'Unnamed: 25', 'Unnamed: 26'], inplace= True)

embolse = embolse.dropna(subset=['Centro'])

#zonas.drop(columns= ['Unnamed: 0'], inplace= True)

In [37]:
# Hay embolses que se dividen en los destinos por lo que hay que separarlos

# Edicion Bauti:
#   /// En el nuevo archivo de embolse no puede verse la presencia del centro 6035.
embolse = embolse[embolse['Centro'] != 6035] #Filtramos el 6035 porque nos dijeron que no iba más

separadas = embolse[(embolse['Centro'] != 6009) & (embolse['Centro'] != 6244)] # & (embolse['Centro'] != 6592)]


In [38]:
# Para los embolses que se mandan a ambos centros en distintas proporciones, armo una linea para cada centro

if len(separadas) > 0:
    separadas['Centro'] = separadas['Centro'].astype(str)
    for ind, row in separadas.iterrows():

            copia = row

            div1, centro1, div2, centro2 = row['Centro'].split(' ')

            if centro1 == '6009':

                centro2 = '6244'

            else: 

                centro1 = '6244'

            embolse.iloc[ind, embolse.columns.get_loc('Centro')] = centro1

            embolse.iloc[ind, embolse.columns.get_loc('SSU estimados')] *= (float(div1[0:2]) / 100)

            copia['Centro'] = centro2

            copia['SSU estimados'] *= (float(div2[0:2]) / 100)

            embolse = embolse.append(copia)


    embolse['Centro'] = embolse['Centro'].replace({'6244': 6244, '6009': 6009})


#Me quedo con las Filas del embolse que estan entre el primer dia y el ultimo a optimizar
embolse['Fecha disponibilidad'] = pd.to_datetime(embolse['Fecha disponibilidad'])

embolse = embolse[(embolse['Fecha disponibilidad'] >= fecha_de_hoy) 
                  & (embolse['Fecha disponibilidad'] <= ultimo_dia)]

embolse = embolse[['Híbridos', 'Unidad', 'SSU estimados', 'Fecha disponibilidad', 'Centro']]

embolse = embolse.groupby(['Híbridos','Fecha disponibilidad']).sum(numeric_only=True).reset_index()

In [39]:
print(embolse.shape)
embolse.sample(3)

(62, 4)


,Híbridos,Fecha disponibilidad,SSU estimados,Centro
5,2773TRE,2023-04-10 15:26:29.760,0.0,6244.0
4,2773TRE,2023-03-19 14:06:43.200,3114.0,6244.0
52,LT721RR2,2023-04-01 10:10:33.600,9000.0,6009.0


In [40]:
prop.sample(3)

,HY,Material,SSU,PROPORCION_DEL_HY
382,DK73-10RR2,12383084,28.495726,0.056561
434,DK73-20VT3P_RIB,12732063,64.150485,0.070933
94,DK69-10RVT3P_RIB,12820568,265.524752,0.172255


In [41]:
def estimar_SKU(hy, ssu_total, prop, disponibilidad, destino): # prop es el dataframe que esta metido en el csv "proporcion_de_material_por_hibrido"
    df = prop[['HY','Material','PROPORCION_DEL_HY']]
    df = df[df['HY'] == hy]
    df['SSU_A_OBTENER'] = (ssu_total* df['PROPORCION_DEL_HY']).round()
    df['Fecha disponibilidad'] = disponibilidad
    df['DESTINOS'] = destino
    return df

In [42]:
def estimar_embolse(embolse, prop):
    df = pd.DataFrame()
    for x in range(len(embolse)):
        data = estimar_SKU(embolse.iloc[x,0], embolse.iloc[x,2], prop, embolse.iloc[x,1],embolse.iloc[x,3])
        #df = df.append(data, ignore_index= True) 
        # Edicion Bauti:
        #   /// Cambio append por concat, ya que append se va a volver obsoleto.
        df = pd.concat([df, data], ignore_index=True)
    return df

In [43]:
# ACA PODRÍA PRIMERO SEPARAR POR FECHA Y DESPUES HACER EL IF

if len(embolse) != 0:
    
    embolse['Fecha disponibilidad'] = pd.to_datetime(embolse['Fecha disponibilidad']).dt.date    
    
    embolse_estimado = estimar_embolse(embolse, prop)
    
    mat_embolse = embolse_estimado['Material']
    
    ssu_embolse = embolse_estimado['SSU_A_OBTENER']
    
    destinos = embolse_estimado['DESTINOS']
    
    fechas_embolse = pd.to_datetime(embolse_estimado['Fecha disponibilidad'])
    
    hibridos_embolse = embolse_estimado['HY']
        
    yq10_embolse = pd.DataFrame({'HY': hibridos_embolse, 'MATERIAL': mat_embolse,'CENTRO': destinos, 'SSU' : ssu_embolse, 'FECHA_DISPONIBILIDAD': fechas_embolse, 'PRIORIDAD_FEFO': 17 })

else:
    
    yq10_embolse = pd.DataFrame(columns=['HY','MATERIAL','CENTRO', 'SSU', 'FECHA_DISPONIBILIDAD', 'PRIORIDAD_FEFO'])

## YQ10

In [44]:
yq10.columns

Index(['Cultivo', 'Material', 'Denominación', 'Lote', 'DAV Number 1 Argentina',
       'DAV Number 2 Argentina', 'Ce.', 'Alm.', 'NºA', 'Denomin.', 'Tp.',
       'Ubicación', 'Lote insp.', 'IL Plant', 'Cód.', 'Fecha DE', 'UMB',
       'LibrUtiliz', 'No libre', 'Bloqueado', 'Trasl.', 'Total',
       'Placa de Siembra', 'Bolsas por pallet',
       'SG Peso neto promedio p. bolsa', 'SG Comentarios sobre el lote',
       'SG Comentarios sobre el lote2'],
      dtype='object')

In [45]:
#Elimino las lineas que no tienen habilitacion DAV 1 ni DAV 2

yq10.dropna(subset=['DAV Number 1 Argentina', 'DAV Number 2 Argentina'], how='all', inplace=True)


yq10 = yq10[((yq10['Ce.'] == 6009) | (yq10['Ce.'] == 6244) | (yq10['Ce.'] == 6592)) & (yq10['Alm.'] == 'WM01')]

#yq10 = yq10[(yq10['Cód.'] == 'LIB1') | (yq10['Cód.'] == 'LIB2') | (yq10['Cód.'] == 'LIB3') 
#            | ((yq10['Cód.'] == 'DETA') & ((pd.to_datetime(yq10['Fecha DE']).dt.day > 15)) & (pd.to_datetime(yq10['Fecha DE']).dt.month >= 10) )]

# Edicion Bauti:
#   /// Le di formato a la fecha, ya que me arrojaba mensaje de advertencia a la hora de usar el dia o el mes, no se podia asociar la posicion de los mismos.
mask_1 = (yq10['Cód.'] == 'LIB1') | (yq10['Cód.'] == 'LIB2') | (yq10['Cód.'] == 'LIB3') 
mask_2 = ((yq10['Cód.'] == 'DETA') & ((pd.to_datetime(yq10['Fecha DE'], format='%d.%m.%Y').dt.day > 15)) & (pd.to_datetime(yq10['Fecha DE'], format='%d.%m.%Y').dt.month >= 10) )
yq10 = yq10[ mask_1 | mask_2 ]

yq10 = yq10[
            (yq10['Ubicación'].str[0:2] != 'RO') & 
            (yq10['Ubicación'].str[0:3] != 'MAN') & 
            (yq10['Ubicación'] != 'STORECV') & 
            (yq10['Ubicación'] != 'VIRTUAL') & 
            (yq10['Ubicación'] != 'MFDENTRG') & 
            (yq10['Ubicación'] != 0)
        ]

yq10 = yq10[yq10['UMB'] == 'BOL']

In [46]:
lotes_habilitados = list(np.sort(yq10['Lote'].str[0].unique()))

#SI AL LLEGAR A LA Z VUELVE A EMPEZAR POR LA A, ESTE CODIGO NO FUNCIONA BIEN
# Edicion Bauti:
#   /// Aca se me ocurre darle valor a las letras y de esa manera cuando llega a la AA va a tener un valor numerico
#   /// y va a ser mas facil filtrar.
#   /// Ademas cambie el '>' por '>=' ya que en el input dice que la campaña especificada es considerada.

#yq10 = yq10[(yq10['Lote'].str[0] > ultima_campaña_despachable)]
yq10 = yq10[(yq10['Lote'].str[0] >= ultima_campaña_despachable)]

In [47]:
yq10.reset_index(drop= True, inplace=True)
'''
yq10['HY'] = ''
for idx, row in yq10.iterrows():
    yq10['HY'].iloc[idx] = row['Denominación'].split(' ')[2]
'''
# Edicion Bauti:
#   /// La sentencia anterior me arroja un mensaje de advertencia, diciendo que se esta tratando de crear una copia sobre un slice del DataFrame.
#   /// Procedo a modificarla, en vez de recorrer la columna 'Denominación' con un 'for', recorro con un 'apply'.
yq10['HY'] = yq10['Denominación'].apply(lambda x: x.split(' ')[2])

In [48]:
def pasar_a_ssu(yq10_):
    
    equivSSU = 80
    
    df1 = yq10_.copy(deep=True)
    '''
    df1['TamanioBolsa'] = 1
    for idx, row in df1.iterrows():
        df1['TamanioBolsa'].iloc[idx] = row['Denominación'].split(' ')[4][:2]
    '''   
    # Edicion Bauti:
    #   /// Nuevamente en vez de un for, utilizo un apply.
    df1['TamanioBolsa'] = df1['Denominación'].apply(lambda x: x.split(' ')[4][:2])
    
    df1 = df1[((df1['TamanioBolsa'] == '80') | ( df1['TamanioBolsa'] == '60') | (df1['TamanioBolsa'] == '40'))]
    df1['TamanioBolsa'] = df1['TamanioBolsa'].astype(int)
    
    '''
    Toma de la yq10 y la pasa a tarima o pallet. LibrUtiliz son las cantidades de bolsas que tengo.
    Aca lo que hace es normalizar segun bolsa de 80 kg.
    Hace una regla de 3 para saber cuanto le va a ocupar en 'SSU' una bolsa de determinado tamaño.
    80 -------------- 1 Unidad 'LibrUtiliz'
    'TamanioBolsa' --- x 
    '''
    df1['SSU'] = df1['LibrUtiliz'] * (df1['TamanioBolsa'] / equivSSU) 
    
    return df1

In [49]:
yq10.reset_index(drop=True, inplace=True)
yq10 = pasar_a_ssu(yq10)

In [50]:
yq10.columns

Index(['Cultivo', 'Material', 'Denominación', 'Lote', 'DAV Number 1 Argentina',
       'DAV Number 2 Argentina', 'Ce.', 'Alm.', 'NºA', 'Denomin.', 'Tp.',
       'Ubicación', 'Lote insp.', 'IL Plant', 'Cód.', 'Fecha DE', 'UMB',
       'LibrUtiliz', 'No libre', 'Bloqueado', 'Trasl.', 'Total',
       'Placa de Siembra', 'Bolsas por pallet',
       'SG Peso neto promedio p. bolsa', 'SG Comentarios sobre el lote',
       'SG Comentarios sobre el lote2', 'HY', 'TamanioBolsa', 'SSU'],
      dtype='object')

In [51]:
yq10.loc[yq10['TamanioBolsa'] == 60,['HY', 'Denominación', 'LibrUtiliz', 'TamanioBolsa', 'SSU']]

,HY,Denominación,LibrUtiliz,TamanioBolsa,SSU
40,DK77-10VT3P,C DK DK77-10VT3P C1 60M TB AR,2.0,60,1.50
41,DK77-10VT3P,C DK DK77-10VT3P C1 60M TB AR,1130.0,60,847.50
42,DK77-10VT3P,C DK DK77-10VT3P C1 60M TB AR,1151.0,60,863.25
43,DK77-10VT3P,C DK DK77-10VT3P C1 60M TB AR,628.0,60,471.00
44,DK77-10VT3P,C DK DK77-10VT3P C2 60M TB AR,0.0,60,0.00
...,...,...,...,...,...
1680,DK77-02TRE,C DK DK77-02TRE R2 60M TB AR,6.0,60,4.50
1681,DK77-02TRE,C DK DK77-02TRE R2 60M TB AR,0.0,60,0.00
1682,DK77-02TRE,C DK DK77-02TRE R2 60M TB AR,0.0,60,0.00
1683,DK77-02TRE,C DK DK77-02TRE R4 60M TB AR,0.0,60,0.00


In [52]:
yq10_final = pd.DataFrame({'HY': yq10['HY'], 'MATERIAL': yq10['Material'],'CENTRO': yq10['Ce.'], 'SSU' : yq10['SSU'], 
                           'FECHA_DISPONIBILIDAD': fecha_de_hoy, 'DE': yq10['Cód.'], 'AÑO_MAX_CAMP': pd.to_datetime(yq10['Fecha DE'], format='%d.%m.%Y').dt.year, 'Campaña': yq10['Lote'].str[0],
                           'Lote': yq10['Lote'], 'Ubicación': yq10['Ubicación'] , 'Peso_bolsa': yq10['SG Peso neto promedio p. bolsa'] })


yq10_final = yq10_final.merge(prioridades, how='left', on= ['DE', 'Campaña', 'AÑO_MAX_CAMP'])

#Hago esto porque las prioridades no tienen en cuenta la campaña U

yq10_final['Prioridad'].fillna(16, inplace= True)


# Agrego distintivo para materiales provenientes del embolse

yq10_final['Campaña'] = yq10_final['Campaña'].fillna('Embolse')

yq10_final.rename(columns={'Prioridad': 'PRIORIDAD_FEFO'}, inplace= True)

yq10_final = yq10_final.drop(columns=['AÑO_MAX_CAMP'])


if len(yq10_embolse) != 0:
    #yq10_final = yq10_final.append(yq10_embolse, ignore_index= True)
    # Edicion Bauti:
    #   /// Cambio append por concat, ya que append va a ser obsoleto.
    #   /// Pregunta: por que se quiere concatenar yq10_final con yq10_embolse??
    #   /// Respuesta: Concatena ambos Df's ya que considera lo que se va a embolsar en un futuro, ya esta disponible en la "actualidad"
    yq10_final = pd.concat([yq10_final, yq10_embolse], ignore_index=True)


# yq10_final['PRIORIDAD_FEFO'].replace([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17], [17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1], inplace= True)
# Edicion Bauti:
#   /// Ya que esto esta hardcodeado y podria haber una prioridad mayor a 17, modifico el codigo.
#   /// En 'PRIORIDAD_FEFO', una prioridad = 1 es la mas alta prioridad, pero, como se trata de un problema de maximizacion y la prioridad es un escalar, 
#   /// al momento de multiplicar este escalar por algun valor se necesita que el valor sea lo mas grande. Entonces en el problema de maximizacion
#   /// una prioridad de 17 implicaria una alta prioridad.
prioridad = list(range(1,int(yq10_final['PRIORIDAD_FEFO'].unique().max() + 1)))
prioridad_inversa = prioridad[::-1]
yq10_final['PRIORIDAD_FEFO'].replace(prioridad, prioridad_inversa,inplace=True)

# Normalizo la prioridad FEFO a valores entre 0 y 1.
yq10_final['PRIORIDAD_FEFO'] = (yq10_final['PRIORIDAD_FEFO'] - 1) / (yq10_final['PRIORIDAD_FEFO'].unique().max() - 1)

yq10_final['FECHA_DISPONIBILIDAD'] = pd.to_datetime(yq10_final['FECHA_DISPONIBILIDAD'])

yq10_final['DE'].fillna(value= 'Emb', inplace= True)

print(yq10_final.shape)
yq10_final.head()


(1966, 11)


,HY,MATERIAL,CENTRO,SSU,FECHA_DISPONIBILIDAD,DE,Campaña,Lote,Ubicación,Peso_bolsa,PRIORIDAD_FEFO
0,LT626VT3P,11495878,6009.0,1.0,2023-02-10,LIB1,T,TE1080979,G11L070A,"15,71",0.5
1,LT626VT3P,11495878,6009.0,0.0,2023-02-10,LIB3,T,TE1082930,GUARDA,"14,62",1.0
2,LT626VT3P,11495878,6009.0,0.0,2023-02-10,LIB3,T,TE1082955,GUARDA,"14,25",1.0
3,DK72-10VT3P,11762933,6009.0,188.0,2023-02-10,LIB3,W,WE1170029D,G11L036B,"21,30",0.5
4,DK72-10VT3P,11762934,6009.0,1296.0,2023-02-10,LIB3,W,WE1170010,G11L096C,"19,02",0.5


In [53]:
# ME FIJO QUE NO SE DUPLIQUE EL STOCK
len(yq10) + len(yq10_embolse) == len(yq10_final)

True

In [54]:
yq10_nueva = yq10_final.groupby(['MATERIAL'])['SSU'].sum()
yq10_nueva = pd.DataFrame({'Material' : yq10_nueva.index, 'Cantidad_Total' : yq10_nueva.values})
yq10_nueva.sample(5)

,Material,Cantidad_Total
260,30116191,2.0
154,30004723,10.0
155,30004724,63.0
8,11762939,611.0
52,12732083,2078.0


## Prioridades

In [55]:
capacidad_despacho_por_dia

12500

In [56]:
def asignar_fechas_entrega(ytr):
    # Esta funcion agarra pedidos historicos pendientes de entrega y le asigna nuevas fechas.
    asignados = ytr[(ytr['Lote'].isna()) & (ytr['Fe_SM_real'].isna())]
    # Edicion Bauti:
    #   /// Pregunta: Por que divide la capacidad por 2?
    #   /// Respuesta: debe ser porque se asignan dos turnos al dia.
    #   /// Pregunta: Podria ser que en la tabla YVATR haya pedidos que no fueron entregados?
    #   /////// de ser asi se les da la maxima prioridad a estos envios pendientes?
    maximo = capacidad_despacho_por_dia / 2
    dia = 1
    suma = 0
    if(len(asignados) > 0):
        for indice in asignados.index.to_list():
            asignados.loc[indice, 'Doc_venta'] = str(asignados.loc[indice, 'Doc_venta']) + '-' + str(indice)
            if (suma + asignados.loc[indice, 'Ctd_entr']) < maximo:
                suma += asignados.loc[indice, 'Ctd_entr']
                asignados.loc[indice, 'Fe_entrega'] = proximo_dia_habil(fecha_de_hoy, dia)
            else:
                suma = asignados.loc[indice, 'Ctd_entr']
                dia += 1
                asignados.loc[indice, 'Fe_entrega'] = proximo_dia_habil(fecha_de_hoy, dia)
    
        #asignados.rename(columns={'  Ctd.fact.': ' Ctd.fact.', '  Ctd.conf.': ' Ctd.conf.',
        #                          '  Ctd.entr.': ' Ctd.entr.', 'Fe-SM real': 'Fe.SM real'}, inplace=True)
        asignados = asignados.copy()
        asignados.loc[:, 'Status'] = 'Liberado'
        # Fuerza la prioridad mas alta para sacarse de encima estos pedidos pendientes
        asignados.loc[:, 'Prioridad'] = 500000
        asignados.loc[:, 'ERROR_FECHA_ENTREGA'] = 'NO'
        asignados.loc[:, 'Recepcion'] = 'Recibe'
        asignados = asignados[['Doc_venta', 'ClVt', 'Solic', 'Nomb_Dest_Mcia', 'GClt', 'Dest_Mcia', 'Oficina_de_Venta',
           'Material', 'SSU', 'Fe_entrega', 'Recepcion','ERROR_FECHA_ENTREGA', 'Status', 'FechaFact', 'FePrefEnt_Cab', 'BqEn', 'Prioridad']]
        # Edicion Bauti:
        #   /// Comento el cambio de nombre a la columna, ya que cuando se realiza el append
        #   /// entre el DF 'despachos' y 'yvaDK' las columnas estas difieren en el nombre
        #   /// lo que provoca un error de concatenacion.
        # asignados.rename(columns={'Nomb_Dest_Mcia': 'Cliente'}, inplace=True)
    return asignados

In [57]:
# Funciones para "calcular" prioridades
anio_actual = fecha_de_hoy.year

anio_anterior = anio_actual - 1

prio1 = 100000

prio2 = 10000

prioridad_pedido = 1000

def obtener_prioridad(potencia):
    return prioridad_pedido / (2 ** potencia)


# REGLA DE LAS FILAS 3 Y 4 (Tipo Pedido)
def prioridad_Clvt(df):
    regla1_tp = (df['ClVt'] == 'ZMKG') | (df['ClVt'] == 'ZBKG')
    regla2_tp = (df['ClVt'] == 'ZROG') | (df['ClVt'] == 'ZCHG')
    regla3_tp = (df['ClVt'] == 'ZSCG') | (df['ClVt'] == 'ZRQG') | (df['ClVt'] == 'ZVQG') 
    
    df.loc[:,'Prioridad_Clvt'] = 0
    df.loc[regla1_tp, 'Prioridad_Clvt'] = prio1
    df.loc[regla2_tp, 'Prioridad_Clvt'] = prio2
    df.loc[regla3_tp, 'Prioridad_Clvt'] = obtener_prioridad(1)
    
    return df


# REGLA DE LA FILA 5 (Fecha Factura)
def prioridad_ff(df):
    regla1_ff = df['FechaFact'].dt.year == anio_anterior
    regla2_ff = df['FechaFact'].dt.year == anio_actual
    regla3_ff = df['FechaFact'].dt.month <= 4
    regla4_ff = df['FechaFact'].dt.month > 4
    
    df.loc[:,'Prioridad_ff'] = 0
    df.loc[regla1_ff, 'Prioridad_ff'] = obtener_prioridad(2)
    df.loc[regla2_ff & regla3_ff, 'Prioridad_ff'] = obtener_prioridad(3)
    df.loc[regla2_ff & regla4_ff, 'Prioridad_ff'] = obtener_prioridad(4)
    
    return df


#REGLA FILA 6 (Tipo Cliente)
def prioridad_tc(df):
    regla1_tc = (df['GClt'] == 'EK') & (df['GClt'] == 'ES')
    regla2_tc = (df['GClt'] == 'EL') & (df['GClt'] == 'E5')
    regla3_tc =df['GClt'] == 'EF'
    regla4_tc =df['GClt'] == 'E2'
    regla5_tc = df['GClt'] == 'EC'
    regla6_tc = df['GClt'] == 'E7'
    regla7_tc = df['GClt'] == 'EY'
    
    df.loc[:, 'Prioridad_tc'] = 0
    df.loc[regla1_tc, 'Prioridad_tc'] = obtener_prioridad(5)
    df.loc[regla2_tc, 'Prioridad_tc'] = obtener_prioridad(6) 
    df.loc[regla3_tc, 'Prioridad_tc'] = obtener_prioridad(7)
    df.loc[regla4_tc, 'Prioridad_tc'] = obtener_prioridad(8)
    df.loc[regla5_tc, 'Prioridad_tc'] = obtener_prioridad(9)
    df.loc[regla6_tc, 'Prioridad_tc'] = obtener_prioridad(10)
    df.loc[regla7_tc, 'Prioridad_tc'] = obtener_prioridad(11)
    
    return df


# REGLA DE LA FILA 7 (Fecha Deseada)
# Edicion Bauti:
#   /// Verificar los limites de los meses.
#   /// 14/2/2023 - Cambie los limites inferiores, para que no se crucen las prioridades.
def prioridad_fd(df):
    regla1_fd = df['FePrefEnt_Cab'].dt.year == anio_anterior
    regla2_fd = df['FePrefEnt_Cab'].dt.year == anio_actual
    regla3_fd = df['FePrefEnt_Cab'].dt.month <= 4
    regla4_fd = df['FePrefEnt_Cab'].dt.month.between(4, 7, inclusive='left')
    regla5_fd = df['FePrefEnt_Cab'].dt.month.between(7, 10, inclusive='left')
    regla6_fd = df['FePrefEnt_Cab'].dt.month > 10
    
    df.loc[:, 'Prioridad_fd'] = 0
    df.loc[ regla1_fd, 'Prioridad_fd'] = obtener_prioridad(12)
    df.loc[ (regla2_fd & regla3_fd), 'Prioridad_fd'] = obtener_prioridad(13)
    df.loc[ (regla2_fd & regla4_fd), 'Prioridad_fd'] = obtener_prioridad(14)
    df.loc[ (regla2_fd & regla5_fd), 'Prioridad_fd'] = obtener_prioridad(15)
    df.loc[ (regla2_fd & regla6_fd), 'Prioridad_fd'] = obtener_prioridad(16)
    
    return df


def prioridad_yva(df):
    df = prioridad_Clvt(df)
    df = prioridad_ff(df)
    df = prioridad_tc(df)
    df = prioridad_fd(df)
    df['Prioridad'] = df['Prioridad_Clvt'] + df['Prioridad_fd'] + df['Prioridad_ff'] + df['Prioridad_tc']
    df.drop(columns=['Prioridad_Clvt', 'Prioridad_fd', 'Prioridad_ff', 'Prioridad_tc'], inplace= True)
    
    return df

## YVA (DATA)

In [58]:
# Obtengo las fechas de Inicio y Fin de Siembra temprana y tardía
# Primero 
feZonas = zonas[['Sales Office description', 'Inicio ', 'Fin', 'Inicio .1', 'Fin.1']].copy()

feZonas.rename(
    columns= {
        'Sales Office description': 'Oficina_de_Venta', 
        'Inicio ':'Inicio Siembra Temprana',
        'Fin':'Fin Siembra Temprana', 
        'Inicio .1':'Inicio Siembra Tardia',
        'Fin.1':'Fin Siembra Tardia'
        },
    inplace=True
    )
feZonas.sample(5)

,Oficina_de_Venta,Inicio Siembra Temprana,Fin Siembra Temprana,Inicio Siembra Tardia,Fin Siembra Tardia
104,S-RIO IV &S.LUIS 4-C,2021-09-05,2021-10-28,2021-11-12,2022-01-10
16,N - ESTE 2 - C,2021-07-31,2021-10-05,2021-11-30,2022-02-04
33,N - NOA 3 - C,2021-08-26,2021-10-15,2021-11-30,2022-01-25
71,S - OESTE 4 - I,2021-08-31,2021-10-25,2021-10-31,2021-12-25
27,N - NEA 3 - I,NaT,NaT,2021-11-16,2022-02-10


In [59]:
# Obtengo la fecha a partir de la cuál cada centro puede recibir mercadería

recep2 = recep[['ID', 'Recepción de bolsas']].copy()

recep2.rename(columns={'ID': 'Dest_Mcia', 'Recepción de bolsas': 'Fecha Recepcion'}, inplace= True)
recep2

,Dest_Mcia,Fecha Recepcion
0,1866447,2022-06-01
1,3624508,2022-06-01
2,1799035,NaT
3,1806801,NaT
4,1806800,2022-10-01
...,...,...
189,1819598,2022-06-01
190,3670347,2022-06-01
191,3457160,NaT
192,3775657,NaT


In [60]:
# Agrego a la YVA las fechas de siembra y de recepción

data.rename(columns= {'Fe-entrega': 'Fe_entrega'},inplace=True)

data = data.merge(feZonas, how='left', on= 'Oficina_de_Venta')

data = data.merge(recep2, how='left', on= 'Dest_Mcia')

In [61]:
data['Fecha Recepcion'].isnull().value_counts()

False    3052
True     1003
Name: Fecha Recepcion, dtype: int64

In [62]:
# Completamos las fechas de recepción vacias y cambiamos las que no reciben

recibe_siempre = pd.to_datetime(str(fecha_de_hoy.year) + '-01-01')

no_recibe = pd.to_datetime(str(fecha_de_hoy.year) + '-12-31')

# Si estan en nulo reciben siempre entoces lleno con 1 de enero

data['Fecha Recepcion'].fillna(value= recibe_siempre, inplace=True)

data['Recepcion'] = data['Fecha Recepcion']

# Los que por alguna razon no reciben, tienen la razón en la columna, por lo que lo cambio
# a fin de año.

data['Fecha Recepcion'] = pd.to_datetime(data['Fecha Recepcion'], errors='coerce')

reciben = data.loc[data['Fecha Recepcion'].notna(),'Fecha Recepcion'].index.to_list()

data.loc[reciben,'Recepcion'] = 'Recibe'

data['Fecha Recepcion'].fillna(no_recibe, inplace= True)

In [63]:
# Marco los pedidos que tienen error en la fecha de entrega = Fecha de entrega < Fecha de Recepción

data['ERROR_FECHA_ENTREGA'] = 'NO'

mask_entrega_mayor_siembraT = pd.to_datetime(data['Fe_entrega']) > pd.to_datetime(data['Inicio Siembra Temprana'])
mask_entrega_menor_recepcion = pd.to_datetime(data['Fe_entrega']) < pd.to_datetime(data['Fecha Recepcion'])

data.loc[mask_entrega_mayor_siembraT | mask_entrega_menor_recepcion,'ERROR_FECHA_ENTREGA'] = 'SI'

# Separo aquellos pedidos flagueados con error en fecha de entrega porque en el output los voy a mostrar separados para que sean analizados

error_fecha = data[data['ERROR_FECHA_ENTREGA'] == 'SI']

error_fecha = error_fecha[['Doc_venta', 'Dest_Mcia', 'Material', 'SSU', 'Fe_entrega', 'Inicio Siembra Temprana', 'Fin Siembra Temprana',
                           'Inicio Siembra Tardia', 'Fin Siembra Tardia', 'Recepcion', 'Fecha Recepcion', 'Status','BqEn', 'ERROR_FECHA_ENTREGA']]

# Edicion Bauti:
#   /// Probar que pasa si cambio sentencia de not a nada.
if not(calcular_fechas_erroneas):           
    
    data = data[data['ERROR_FECHA_ENTREGA'] != 'SI']
    
else: 
    
    #data['Fe_entrega'][pd.to_datetime(data['Fe_entrega']) > pd.to_datetime(data['Inicio Siembra Temprana'])] = pd.to_datetime(data['Inicio Siembra Temprana']) - pd.Timedelta(value= 7, unit='D')
    data.loc[mask_entrega_mayor_siembraT, 'Fe_entrega'] = pd.to_datetime(data['Inicio Siembra Temprana']) - pd.Timedelta(value= 7, unit='D')    

In [64]:
yvaDK = data[['Doc_venta', 'ClVt', 'Solic', 'Nomb_Dest_Mcia', 'GClt', 'Dest_Mcia', 'Material', 'SSU', 'Oficina_de_Venta', 'Fe_entrega', 'Inicio Siembra Temprana', 'Fin Siembra Temprana', 'Inicio Siembra Tardia', 'Fin Siembra Tardia', 'Recepcion', 'Fecha Recepcion', 'FechaFact', 'FePrefEnt_Cab', 'Status','BqEn', 'ERROR_FECHA_ENTREGA']]

# Edicion Bauti:
#   /// Ya que se puede ver pedidos con SSU nulo, procedere a filtrarlos, asi no traen problemas.
yvaDK_ssu_nulo = yvaDK[yvaDK['SSU'] == 0]
yvaDK = yvaDK[yvaDK['SSU'] != 0]

if USAR_FECHA_SIEMBRA_TEMPRANA:
    yvaDK['Fe_entrega'] = yvaDK['Inicio Siembra Temprana']
    
    #ultimo_dia = yvaDK['Fin Siembra Temprana'].max() -> Comento esta linea porque genera que entren todos los pedidos

elif USAR_FECHA_SIEMBRA_TARDIA:
    yvaDK['Fe_entrega'] = yvaDK['Inicio Siembra Tardia']
    
    #ultimo_dia = yvaDK['Fin Siembra Tardia'].max()

else:
    pass

yvaDK = yvaDK.copy()    # Para que no arroje warning.
yvaDK.rename(columns={'Nomb_Dest_Mcia': 'Cliente'}, inplace= True)

In [65]:
yvaDK['Fe_entrega'] = pd.to_datetime(yvaDK['Fe_entrega'])

yvaDK['FePrefEnt_Cab'] = pd.to_datetime(yvaDK['FePrefEnt_Cab'])

yvaDK['FechaFact'] = pd.to_datetime(yvaDK['FechaFact'])

In [66]:
# Filtro del YVA los pedidos que se vencen en el tiempo que se eligio
yvaDK = yvaDK[(yvaDK['Fe_entrega'] >= fecha_de_hoy) & (yvaDK['Fe_entrega'] <= ultimo_dia)]

In [67]:
# Se agrega a la YVA el GClt

# CAMBIO ACÁ
#maestro = maestro[['Solic.', 'GClt']]

#yvaDK = yvaDK.merge(maestro, on='Solic.', how='left')

# Asigno prioridad a los pedidos

yvaDK = prioridad_yva(yvaDK)

despachos = asignar_fechas_entrega(yvatr)

#despachos = despachos.merge(maestro, on='Solic.', how='left')

despachos = despachos.merge(feZonas, how='left', on= 'Oficina_de_Venta')
despachos = despachos.merge(recep2, how='left', on= 'Dest_Mcia')
yvaDK['Status'].unique()

array(['Bloq Fac', 'Liberado'], dtype=object)

In [68]:
despachos['Fecha Recepcion'].fillna(value= fecha_de_hoy, inplace=True)

#Los que tienen fecha de recepción
despachos['Fecha Recepcion'] = pd.to_datetime(data['Fecha Recepcion'], errors='coerce')

In [69]:
#Chequeo que haya envios para que no pinche y concateno

if(len(despachos) > 0):
   '''
   despachos = despachos[['Doc_venta', 'ClVt', 'Solic', 'Cliente', 'GClt', 'Dest_Mcia',
       'Material', 'SSU', 'Oficina_de_Venta', 'Fe_entrega', 'Inicio Siembra Temprana',
       'Fin Siembra Temprana', 'Inicio Siembra Tardia', 'Fin Siembra Tardia', 'Recepcion',
       'Fecha Recepcion', 'Status', 'FechaFact', 'FePrefEnt_Cab', 'BqEn',
       'ERROR_FECHA_ENTREGA', 'Prioridad']]
   '''
   despachos = despachos[['Doc_venta', 'ClVt', 'Solic', 'Nomb_Dest_Mcia', 'GClt', 'Dest_Mcia',
       'Material', 'SSU', 'Oficina_de_Venta', 'Fe_entrega', 'Inicio Siembra Temprana',
       'Fin Siembra Temprana', 'Inicio Siembra Tardia', 'Fin Siembra Tardia', 'Recepcion',
       'Fecha Recepcion', 'Status', 'FechaFact', 'FePrefEnt_Cab', 'BqEn',
       'ERROR_FECHA_ENTREGA', 'Prioridad']]
   
   # Se agregan los despachos a la YVA
   #yvaDK = yvaDK.append(despachos, ignore_index=True)
   yvaDK = pd.concat([yvaDK, despachos], ignore_index=True)

In [70]:
# Dejo las prioridades entre 1 y 10
#yvaDK['Prioridad'] = 1 + ( (( yvaDK['Prioridad'] - yvaDK['Prioridad'].min()) * (10 - 1)) / (yvaDK['Prioridad'].max() - yvaDK['Prioridad'].min()))

In [71]:
# Si hay un pedido pendiente con fecha pasada, se le reasigna la fecha de entrega al último día de los que tomé
yvaDK.loc[yvaDK['Fe_entrega'] <= datetime.datetime.today(),'Fe_entrega'] = proximo_dia_habil(fecha_de_hoy, 7)#pd.to_datetime('20/06/2021')

# Hago un nuevo DF con el material en stock del material solicitado en el pedido
yvaDK = yvaDK.merge(yq10_nueva,how= 'left', on='Material')

In [72]:
# Separo los pedidos liberados de los bloqueados ya que se van a correr de forma separada
liberados = yvaDK[yvaDK['Status'] == 'Liberado'].copy().reset_index(drop=True)
bloqueados = yvaDK[yvaDK['Status'] != 'Liberado'].copy().reset_index(drop=True)

# Cambio los NaN por ceros
liberados.fillna(value= {'Cantidad_Total': 0} , inplace= True)

bloqueados.fillna(value= {'Cantidad_Total': 0} , inplace= True)
print(liberados.shape)
liberados.head()

(5, 24)


,Doc_venta,ClVt,Solic,Cliente,GClt,Dest_Mcia,Material,SSU,Oficina_de_Venta,Fe_entrega,...,Recepcion,Fecha Recepcion,FechaFact,FePrefEnt_Cab,Status,BqEn,ERROR_FECHA_ENTREGA,Prioridad,Nomb_Dest_Mcia,Cantidad_Total
0,828568192,ZSCG,3442670,R Y K SRL,EL,1809756,30174172,90.0,N - NEA 3 - I,2023-02-21,...,Recibe,2022-09-01,2023-01-27,2023-02-15,Liberado,NaN,NO,625.12207,NaN,0.0
1,828568192,ZSCG,3442670,R Y K SRL,EL,1809756,30114422,585.0,N - NEA 3 - I,2023-02-21,...,Recibe,2022-09-01,2023-01-27,2023-02-15,Liberado,NaN,NO,625.12207,NaN,0.0
2,828568125,ZSCG,1822121,DEKANOR SRL,EF,3395980,12787691,10.0,N - NEA 3 - I,2023-02-21,...,Recibe,2023-01-01,2023-01-27,2023-02-15,Liberado,NaN,NO,632.93457,NaN,157.0
3,828568125,ZSCG,1822121,DEKANOR SRL,EF,3395980,30174163,90.0,N - NEA 3 - I,2023-02-21,...,Recibe,2023-01-01,2023-01-27,2023-02-15,Liberado,NaN,NO,632.93457,NaN,9.0
4,828384334-16446,ZCHG,1802978,NaN,EL,1806801,30114751,1.0,N - NOA 3 - I,2023-02-21,...,Recibe,2022-07-15,2022-06-16,2022-06-16,Liberado,NaN,NO,500000.00000,TUCAGRO SRL,0.0


#### ----------------------------------------------------COSAS PARA TESTEAR - COMIENZO------------------------------------------------------------------------------

#### --------------------------------------------COSAS PARA TESTEAR - FIN-------------------------------------------------------------------------------------------------

In [73]:
yq10_final = yq10_final[yq10_final['SSU'] >= 0]

yq10_final.reset_index(inplace= True, drop= True)

In [74]:
yq10_final[yq10_final['CENTRO'] == 6244]

,HY,MATERIAL,CENTRO,SSU,FECHA_DISPONIBILIDAD,DE,Campaña,Lote,Ubicación,Peso_bolsa,PRIORIDAD_FEFO
51,DK77-10VT3P,12697648,6244.0,264.00,2023-02-10,LIB3,W,WE1070072G,06009D,"12,34",0.5
52,DK77-10VT3P,12697648,6244.0,12.00,2023-02-10,LIB3,W,WE1170046,06004D,"13,24",0.5
53,DK77-10VT3P,12697648,6244.0,203.25,2023-02-10,LIB3,W,WE1170046,06007R,"13,24",0.5
58,DK77-10VT3P,12697670,6244.0,247.50,2023-02-10,LIB1,W,WE1070044,06005D,"22,37",0.5
59,DK77-10VT3P,12697670,6244.0,270.00,2023-02-10,LIB1,W,WE1070044,06005R,"22,37",0.5
...,...,...,...,...,...,...,...,...,...,...,...
1961,LT720VT3P,12961819,6244.0,0.00,2023-06-28,Emb,NaN,NaN,NaN,NaN,0.0
1962,LT720VT3P,12961872,6244.0,0.00,2023-06-28,Emb,NaN,NaN,NaN,NaN,0.0
1963,LT720VT3P,12961873,6244.0,0.00,2023-06-28,Emb,NaN,NaN,NaN,NaN,0.0
1964,LT720VT3P,12961874,6244.0,0.00,2023-06-28,Emb,NaN,NaN,NaN,NaN,0.0


In [75]:
# Primero voy a agregar registros para frozar ciertas restricciones y envios.
add_1 = pd.DataFrame(
    {
        'Doc_venta' : 828419574,
        'ClVt' : 'ZSCG',
        'Solic' : 1801809,
        'Nomb_Dest_Mcia' : 'Bauti',
        'GClt' : 'BA',
        'Dest_Mcia' : 1801809,
        'Material' : 11789429,
        'SSU' : 4,
        'Oficina_de_Venta' : 'BAU-SE',
        'Fe_entrega' : proximo_dia_habil(fecha_de_hoy, 7),
        'Inicio Siembra Temprana' : 'Nat',
        'Fin Siembra Temprana' : 'Nat',
        'Inicio Siembra Tardia' : 'Nat',
        'Fin Siembra Tardia' : 'Nat',
        'Recepcion' :   'Recibe',
        'Fecha Recepcion' : pd.to_datetime('2022-01-01'),
        'FechaFact' : pd.to_datetime('2022-06-25'),
        'FePrefEnt_Cab' : pd.to_datetime('2022-09-15'),
        'Status' : 'Liberado',
        'BqEn' : np.NaN,
        'ERROR_FECHA_ENTREGA' : 'NO',
        'Prioridad' : liberados.loc[0,'Prioridad'],
        'Cantidad_Total' : yq10_final.loc[yq10_final['MATERIAL'] == 11789429, 'SSU'].sum()
    },
    index = [0]
)
liberados = pd.concat([liberados, add_1], ignore_index=True)

add_2 = pd.DataFrame(
    {
        'Doc_venta' : 828419583,
        'ClVt' : 'ZSCG',
        'Solic' : 1901809,
        'Nomb_Dest_Mcia' : 'Bauti',
        'GClt' : 'BA',
        'Dest_Mcia' : 1901809,
        'Material' : 11789429,
        'SSU' : 1,
        'Oficina_de_Venta' : 'BAU-SE',
        'Fe_entrega' : proximo_dia_habil(fecha_de_hoy, 7),
        'Inicio Siembra Temprana' : 'Nat',
        'Fin Siembra Temprana' : 'Nat',
        'Inicio Siembra Tardia' : 'Nat',
        'Fin Siembra Tardia' : 'Nat',
        'Recepcion' :   'Recibe',
        'Fecha Recepcion' : pd.to_datetime('2022-01-01'),
        'FechaFact' : pd.to_datetime('2022-06-25'),
        'FePrefEnt_Cab' : pd.to_datetime('2022-09-15'),
        'Status' : 'Liberado',
        'BqEn' : np.NaN,
        'ERROR_FECHA_ENTREGA' : 'NO',
        'Prioridad' : liberados.loc[0,'Prioridad'],
        'Cantidad_Total' : yq10_final.loc[yq10_final['MATERIAL'] == 11789429, 'SSU'].sum()
    },
    index = [0]
)
liberados = pd.concat([liberados, add_2], ignore_index=True)

add_3 = pd.DataFrame(
    {
        'Doc_venta' : 828333583,
        'ClVt' : 'ZSCG',
        'Solic' : 1988809,
        'Nomb_Dest_Mcia' : 'Bauti',
        'GClt' : 'BA',
        'Dest_Mcia' : 1988809,
        'Material' : 12324881,
        'SSU' : 6,
        'Oficina_de_Venta' : 'BAU-SE',
        'Fe_entrega' : proximo_dia_habil(fecha_de_hoy, 7),
        'Inicio Siembra Temprana' : 'Nat',
        'Fin Siembra Temprana' : 'Nat',
        'Inicio Siembra Tardia' : 'Nat',
        'Fin Siembra Tardia' : 'Nat',
        'Recepcion' :   'Recibe',
        'Fecha Recepcion' : pd.to_datetime('2022-01-01'),
        'FechaFact' : pd.to_datetime('2022-06-25'),
        'FePrefEnt_Cab' : pd.to_datetime('2022-09-15'),
        'Status' : 'Liberado',
        'BqEn' : np.NaN,
        'ERROR_FECHA_ENTREGA' : 'NO',
        'Prioridad' : liberados.loc[0,'Prioridad'],
        'Cantidad_Total' : yq10_final.loc[yq10_final['MATERIAL'] == 12324881, 'SSU'].sum()
    },
    index = [0]
)
liberados = pd.concat([liberados, add_3], ignore_index=True)

add_4 = pd.DataFrame(
    {
        'Doc_venta' : 82838583,
        'ClVt' : 'ZSCG',
        'Solic' : 1982209,
        'Nomb_Dest_Mcia' : 'Bauti',
        'GClt' : 'BA',
        'Dest_Mcia' : 1982209,
        'Material' : 12950427,
        'SSU' : 5,
        'Oficina_de_Venta' : 'BAU-SE',
        'Fe_entrega' : proximo_dia_habil(fecha_de_hoy, 7),
        'Inicio Siembra Temprana' : 'Nat',
        'Fin Siembra Temprana' : 'Nat',
        'Inicio Siembra Tardia' : 'Nat',
        'Fin Siembra Tardia' : 'Nat',
        'Recepcion' :   'Recibe',
        'Fecha Recepcion' : pd.to_datetime('2022-01-01'),
        'FechaFact' : pd.to_datetime('2022-06-25'),
        'FePrefEnt_Cab' : pd.to_datetime('2022-09-15'),
        'Status' : 'Liberado',
        'BqEn' : np.NaN,
        'ERROR_FECHA_ENTREGA' : 'NO',
        'Prioridad' : liberados.loc[0,'Prioridad'],
        'Cantidad_Total' : yq10_final.loc[yq10_final['MATERIAL'] == 12950427, 'SSU'].sum()
    },
    index = [0]
)
liberados = pd.concat([liberados, add_4], ignore_index=True)

add_5 = pd.DataFrame(
    {
        'Doc_venta' : 82838583,
        'ClVt' : 'ZSCG',
        'Solic' : 1982209,
        'Nomb_Dest_Mcia' : 'Bauti',
        'GClt' : 'BA',
        'Dest_Mcia' : 1982209,
        'Material' : 12950427,
        'SSU' : 1,
        'Oficina_de_Venta' : 'BAU-SE',
        'Fe_entrega' : proximo_dia_habil(fecha_de_hoy, 7),
        'Inicio Siembra Temprana' : 'Nat',
        'Fin Siembra Temprana' : 'Nat',
        'Inicio Siembra Tardia' : 'Nat',
        'Fin Siembra Tardia' : 'Nat',
        'Recepcion' :   'Recibe',
        'Fecha Recepcion' : pd.to_datetime('2022-01-01'),
        'FechaFact' : pd.to_datetime('2022-06-25'),
        'FePrefEnt_Cab' : pd.to_datetime('2022-09-15'),
        'Status' : 'Liberado',
        'BqEn' : np.NaN,
        'ERROR_FECHA_ENTREGA' : 'NO',
        'Prioridad' : liberados.loc[0,'Prioridad'],
        'Cantidad_Total' : yq10_final.loc[yq10_final['MATERIAL'] == 12950427, 'SSU'].sum()
    },
    index = [0]
)
liberados = pd.concat([liberados, add_5], ignore_index=True)

add_6 = pd.DataFrame(
    {
        'Doc_venta' : 590888255,
        'ClVt' : 'ZSCG',
        'Solic' : 1982209,
        'Nomb_Dest_Mcia' : 'Bauti',
        'GClt' : 'BA',
        'Dest_Mcia' : 1982209,
        'Material' : 12697648,
        'SSU' : 25,
        'Oficina_de_Venta' : 'BAU-SE',
        'Fe_entrega' : proximo_dia_habil(fecha_de_hoy, 7),
        'Inicio Siembra Temprana' : 'Nat',
        'Fin Siembra Temprana' : 'Nat',
        'Inicio Siembra Tardia' : 'Nat',
        'Fin Siembra Tardia' : 'Nat',
        'Recepcion' :   'Recibe',
        'Fecha Recepcion' : pd.to_datetime('2022-01-01'),
        'FechaFact' : pd.to_datetime('2022-06-25'),
        'FePrefEnt_Cab' : pd.to_datetime('2022-09-15'),
        'Status' : 'Liberado',
        'BqEn' : np.NaN,
        'ERROR_FECHA_ENTREGA' : 'NO',
        'Prioridad' : liberados.loc[0,'Prioridad'],
        'Cantidad_Total' : yq10_final.loc[yq10_final['MATERIAL'] == 12697648, 'SSU'].sum()
    },
    index = [0]
)
liberados = pd.concat([liberados, add_6], ignore_index=True)

In [76]:
# Edicion Bauti:
#   /// Ya que se exceden las 1000 variables y restricciones que me permite tener la version gratutita, filtro los DataFrames
#   /// para poder hacer una ejecucion del modelo.

# Primero filtro la cantidad de pedidos para no exceder las restricciones.
'''if len(liberados) > 45:
    liberados =liberados.iloc[0:45, :].reset_index(drop=True)
if len(bloqueados) > 40:
    bloqueados = bloqueados.iloc[0:40, :].reset_index(drop=True)'''
    
# Me quedo con los materiales que se entregaran.
mat_a_entregar = liberados['Material'].unique()

# Me quedo con los materiales de pedidos que se encuentran bloqueados.
mat_bloqueados = bloqueados['Material'].unique()

print('Shape antes %s columnas, %s filas' %(yq10_final.shape[1], yq10_final.shape[0]))

#mask_centro = yq10_final['CENTRO'] == 6009
mask_mat_a_entregar = yq10_final['MATERIAL'].isin(mat_a_entregar)
mask_bloqueados = yq10_final['MATERIAL'].isin(mat_bloqueados)

# Filtro la YQ10 para solo quedarme con esos materiales
yq10_final = yq10_final[(mask_mat_a_entregar) | (mask_bloqueados)].reset_index(drop=True)

print('Shape antes %s columnas, %s filas' %(yq10_final.shape[1], yq10_final.shape[0]))

yq10_final.sample(3)

Shape antes 11 columnas, 1966 filas
Shape antes 11 columnas, 161 filas


,HY,MATERIAL,CENTRO,SSU,FECHA_DISPONIBILIDAD,DE,Campaña,Lote,Ubicación,Peso_bolsa,PRIORIDAD_FEFO
40,722RR2,12950427,6009.0,724.0,2023-02-10,LIB3,W,WE1072821N,G13L070B,"21,95",0.5
59,DK72-20PRO4,30114751,6009.0,0.0,2023-02-10,LIB3,W,WE8870086,PICKING1,"16,76",0.5
103,DK72-20RR2,30116097,6009.0,0.0,2023-02-10,LIB1,V,VE1160141,PICKING1,"14,25",0.5


In [77]:
print(liberados.shape)
liberados.sample(3)

(11, 24)


,Doc_venta,ClVt,Solic,Cliente,GClt,Dest_Mcia,Material,SSU,Oficina_de_Venta,Fe_entrega,...,Recepcion,Fecha Recepcion,FechaFact,FePrefEnt_Cab,Status,BqEn,ERROR_FECHA_ENTREGA,Prioridad,Nomb_Dest_Mcia,Cantidad_Total
5,828419574,ZSCG,1801809,NaN,BA,1801809,11789429,4.0,BAU-SE,2023-02-21,...,Recibe,2022-01-01,2022-06-25,2022-09-15,Liberado,NaN,NO,625.12207,Bauti,6.0
1,828568192,ZSCG,3442670,R Y K SRL,EL,1809756,30114422,585.0,N - NEA 3 - I,2023-02-21,...,Recibe,2022-09-01,2023-01-27,2023-02-15,Liberado,NaN,NO,625.12207,NaN,0.0
0,828568192,ZSCG,3442670,R Y K SRL,EL,1809756,30174172,90.0,N - NEA 3 - I,2023-02-21,...,Recibe,2022-09-01,2023-01-27,2023-02-15,Liberado,NaN,NO,625.12207,NaN,0.0


In [78]:
print(bloqueados.shape)
bloqueados.sample(3)

(28, 24)


,Doc_venta,ClVt,Solic,Cliente,GClt,Dest_Mcia,Material,SSU,Oficina_de_Venta,Fe_entrega,...,Recepcion,Fecha Recepcion,FechaFact,FePrefEnt_Cab,Status,BqEn,ERROR_FECHA_ENTREGA,Prioridad,Nomb_Dest_Mcia,Cantidad_Total
0,828582260,ZSCG,1807731,ENRIQUE M BAYA CASAL SA,EC,3115375,30114759,1300.0,KA6-SE,2023-02-21,...,Recibe,2023-01-01,2023-02-10,2023-02-20,Bloq Fac,36,NO,627.075195,NaN,2151.0
24,828557779,ZSCG,3774633,R Y K SRL,EF,1809756,12787701,8.0,N - NEA 3 - I,2023-02-21,...,Recibe,2022-09-01,2023-01-18,2023-02-15,Bloq Fac,39,NO,632.934570,NaN,836.0
17,828560955,ZSCG,1807510,RAMIREZ JOSE MARIA,EC,1812785,12787701,60.0,N-M1-E,2023-02-21,...,Recibe,2022-05-01,2023-01-16,2023-02-15,Bloq Fac,39,NO,627.075195,NaN,836.0


## CPLEX

In [79]:
# Listas que voy a utilizar para armar las variables que necesita el modelo

indices_yq10 = yq10_final.index.to_list()

yq10_final['Indice_Modelo'] = indices_yq10

pedidos_unicos = liberados['Doc_venta'].unique()

In [80]:
modelo = Model(name="pedidos")

In [81]:
# Una variable binaria por cada pedido que indica si será despachado o no

binarios = modelo.binary_var_dict(pedidos_unicos, name='binaria')

# Una variable discreta por cada material (producto) que indica cuanto se va a despachar
#cant_material_a_despachar = modelo.integer_var_dict(indices_yq10)
cant_material_a_despachar = modelo.continuous_var_dict(indices_yq10, name='cantidad')

In [82]:
modelo.print_information()

Model: pedidos
 - number of variables: 169
   - binary=8, integer=0, continuous=161
 - number of constraints: 0
   - linear=0
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [83]:
yq10_final['SSU'] = (yq10_final['SSU']).astype(int)
yq10_final['CENTRO'] = yq10_final['CENTRO'].astype(int)

## CONSTRAINTS

In [84]:
materiales_pedidos = liberados['Material'].unique()

In [85]:
promedio_despacho = liberados['SSU'].mean()

In [86]:
# La cantidad a despachar de un material no supere la cantidad disponible de ese material
def constraint_1(model,yq10_):
    # Lista de indices de las variables.
    var_ind = list(cant_material_a_despachar.keys())
    for ind in var_ind:
        
        # Un unico material no puede superar la cantidad de stock que se tiene (para ESE material) 
        model.add_constraint(
            cant_material_a_despachar[ind]
            <=
            yq10_['SSU'].iloc[ind]
            )

#%%time
constraint_1(modelo,yq10_final)

modelo.print_information()

Model: pedidos
 - number of variables: 169
   - binary=8, integer=0, continuous=161
 - number of constraints: 161
   - linear=161
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [87]:
# La suma de la cantidad a despachar por cada material sea igual a la cantidad que hay que despachar
# Lo que me piden es igual a lo que voy a despachar.
def constraint_2(model,yq10_,yva_):
    # Array con los materiales unicos en stock.
    materiales_stock = yq10_['MATERIAL'].unique()
    for m in materiales_stock:
        
        # DF con la cantidad que tengo en STOCK del material 'm'
        materiales = yq10_[yq10_['MATERIAL'] == m]
        # DF con la cantidad que tengo que ENVIAR del material 'm'
        yva = yva_[yva_['Material'] == m]
        
        # Array con los id's de venta, del material 'm'.
        pedidos = yva['Doc_venta'].unique()
        # Array con los indices del material 'm' en stock, 
        # se corresponden con los indices de las variables continuas
        lista_indices = materiales.index
        
        # La suma de TODOS los materiales 'm' que tengo en stock y tengo que despachar (variables)
        # tiene que ser IGUAL a la cantidad de material 'm' a enviar en el pedido 'ped'.
        model.add_constraint(
                            model.sum(
                                    cant_material_a_despachar[ind] for ind in lista_indices
                                    )
                            ==
                            model.sum(
                                    binarios[ped] * yva.loc[(yva['Doc_venta'] == ped) & (yva['Material'] == m), 'SSU'].sum() for ped in pedidos
                                    )
                            )
        
#%%time
constraint_2(modelo,yq10_final,liberados)

modelo.print_information()

Model: pedidos
 - number of variables: 169
   - binary=8, integer=0, continuous=161
 - number of constraints: 176
   - linear=176
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [88]:

dias_df = pd.DataFrame({'Fecha' : pd.date_range(fecha_de_hoy, ultimo_dia, freq='D')})

# Constraint para asegurar que no se pueda superar la capacidad de REAP por día.
def constraint_3(model,yq10_,yva_):
    
    yva = yva_.copy(deep=True)
    yq10 = yq10_.copy(deep=True)
    
    # Obtengo los numeros de pedidos unicos (sin repetidos, 
    # puede haber varios pedidos con un numero de pedido igual)
    pedKeys = list(binarios.keys()).copy()
    # Obtengo las variables binarias para cada numero de pedido unico.
    pedVals = list(binarios.values()).copy()
    
    # Obtengo el id (de la yq10) de cada material disponible en cada centro.
    stockKeys = list(cant_material_a_despachar.keys()).copy()
    # Obtengo las variables (cuanto de cada material en stock voy a despachar).
    stockVals = list(cant_material_a_despachar.values()).copy()
    
    # Array que contiene todos los materiales pendientes de entrega.
    materiales_pedidos = yva['Material'].unique()
    
    # DF que almacena un unico numero de pedido con su correspondiente variable.
    dfBin = pd.DataFrame({'Doc_venta': pedKeys, 'Despacho': pedVals})
    
    # DF con los indices de la yq10 (materiales) y con sus correspondientes variables.
    stockCons = pd.DataFrame({'Indice_Modelo': stockKeys, 'Despacho': stockVals})
    
    # Por cada material pendiente de entrega, voy a restar dia a dia lo que voy a ir enviando de lo que tengo en stock.
    for m in materiales_pedidos:
        # Filtro los pedidos que tienen el material 'm'.      
        pedidos_por_material = yva.loc[yva['Material'] == m, ['Fe_entrega', 'Doc_venta', 'SSU', 'Material']]
        # Mergeo los pedidos con las variables binarias.
        pedidos_por_material = pedidos_por_material.merge(dfBin, how='left', on='Doc_venta', copy=False)
        
        pedidos_por_material['Fe_entrega'] = pd.to_datetime(pedidos_por_material['Fe_entrega'])
        
        # Multiplico la variable binaria por la cantidad a despachar.
        pedidos_por_material['cantPorDesp'] = pedidos_por_material['SSU'] * pedidos_por_material['Despacho']
        
        # Materiales pedidos por dia, para el rango de fechas en 'dias_df', va a haber fechas vacias relleno NaN con 0.
        pedidos_mat_dia = pedidos_por_material.merge(dias_df, how='right', left_on = 'Fe_entrega', right_on = 'Fecha').fillna(0)
        
        # Obtengo la cantidad acumulada de materiales a despachar dia a dia.
        dfResult_mat = pedidos_mat_dia[['Fecha', 'cantPorDesp']]
        dfResult_mat = dfResult_mat.groupby('Fecha').sum().reset_index()
        dfResult_mat['cumSumDesp'] = np.cumsum(dfResult_mat['cantPorDesp'])
        
        
        
        # Me quedo con la cantidad del material 'm' que hay disponible en el centro 6009
        mask = (yq10['CENTRO'] == 6009) & (yq10['MATERIAL'] == m)
        stock_por_material = yq10.loc[mask, ['FECHA_DISPONIBILIDAD', 'Indice_Modelo']]
        # Agrego las variables de la cantidad a despachar, de los materiales disponibles en la 6009.
        stock_por_material = stock_por_material.merge(stockCons, how='left', on='Indice_Modelo', copy=False)
        
        # Stock de materiales por dia, para el rango de fechas en 'dias_df', relleno nulos.
        stock_mat_dia = stock_por_material.merge(dias_df, how='right', left_on = 'FECHA_DISPONIBILIDAD', right_on = 'Fecha').fillna(0)
        
        # Obtengo las cantidades acumuladas en stock dia a dia.
        dfResult_stock = stock_mat_dia[['Fecha', 'Despacho']]
        dfResult_stock = dfResult_stock.groupby('Fecha').sum().reset_index()
        dfResult_stock['cumSumStock'] = np.cumsum(dfResult_stock['Despacho'])
        
        # Dataframe en donde me va a figurar la cantidad de material que tengo en stock y la que tengo que despachar para cada dia
        dfResult = dfResult_stock[['Fecha', 'cumSumStock']].merge(dfResult_mat[['Fecha', 'cumSumDesp']], how='left', on='Fecha', copy=False)
        # Edicion Bauti:
        #   /// Pregunta: Esta bien que se reste lo que tengo en stock a lo que tengo que despachar, no seria al revez?
        dfResult['Resta'] = dfResult['cumSumDesp'] - dfResult['cumSumStock']
    
    dfResult = dfResult.groupby('Fecha').sum().reset_index()
    
    for i in dfResult.index.to_list():
        # Cuento el numero de dias habiles entre la fecha de que se hace la optimizacion (valor obtenido del input)
        # y la fecha de entrega.
        difDias = np.busday_count(fecha_de_hoy.date(), pd.to_datetime(dfResult.loc[i, 'Fecha']).date())
        # Obtengo el reaprovisionamiento para el intervalo de fechas considerado.
        reaprov_posible = capacidad_reap_por_dia * difDias
        
        dfResult.loc[i, 'dias_acumulados']= reaprov_posible
        # La diferencia de lo que tengo que enviar y lo que tengo en stock NO PUEDE ser mayor a lo que va a entrar.
        model.add_constraint(
            dfResult.loc[i, 'Resta'] <= reaprov_posible
            )
        
    #return dfResult
#%%time
constraint_3(modelo,yq10_final,liberados)

modelo.print_information()

Model: pedidos
 - number of variables: 169
   - binary=8, integer=0, continuous=161
 - number of constraints: 324
   - linear=324
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [89]:
dias_distintos = (liberados['Fe_entrega'].unique()).astype(str)
dias_distintos = pd.to_datetime(dias_distintos)

# Por cada material, chequea que la suma de las cantidades por pedido de ese material sea menor o igual al stock
def constraint_4(model,yq10_,yva_):
    # Creo array con los pedidos unicos pendientes de entrega.
    materiales_pedidos = yva_['Material'].unique()
    
    # Por cada material.
    for m in materiales_pedidos:
        # Creo un slice del DF con los pedidos que tienen ese material para entregar.
        yva2 = yva_[yva_['Material'] == m]
        
        # Obtengo los distintos dias en los que se van a estar entregando materiales.
        dias_con_pedidos = yva2['Fe_entrega'].unique().astype(str)
        dias_con_pedidos = pd.to_datetime(dias_con_pedidos)
        
        # Para cada dia de entrega me fijo el stock disponible y los pedidos a realizar.
        for dia in dias_con_pedidos:
            # Creo slice de los pedidos pendientes de entrega considerando las entregas del dia del loop y las anteriores.
            # Esto se hace para considerar las entregas que se realizaron previamente a la fecha ('dia')
            # asi de esa manera tengo en cuenta cuanta cantidad del material 'm' ya se envio y se desconto del stock.
            yva_fecha = yva2[yva2['Fe_entrega'] <= dia]
            
            # Pedidos realizados en las fechas consideradas.
            pedidos_unicos = yva_fecha['Doc_venta'].unique()
            
            # Veo el stock disponible del material 'm' hasta el dia considerado.
            mask1 = yq10_['FECHA_DISPONIBILIDAD'] <= dia
            mask2 = yq10_['MATERIAL'] == m 
            stock_yq10_fecha = yq10_[mask1 & mask2]
            
            # Variables de los materiales en stock.
            variables_disp = stock_yq10_fecha['Indice_Modelo'].unique()
            
            # La suma de los materiales a despachar para el periodo de fecha considerado tiene que ser menor o igual
            # a el stock disponible en dichas fechas.
            model.add_constraint(
                model.sum(
                        cant_material_a_despachar[var] for var in variables_disp
                        )
                >=
                model.sum(
                        binarios[nro_pedido] * (yva_fecha.loc[yva_fecha['Doc_venta'] == nro_pedido, 'SSU']).sum() for nro_pedido in pedidos_unicos
                        )
            )

#%%time
constraint_4(modelo,yq10_final,liberados)

modelo.print_information()

Model: pedidos
 - number of variables: 169
   - binary=8, integer=0, continuous=161
 - number of constraints: 333
   - linear=333
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [90]:
# El problema de esta constraint es que: por ahi se descartan pedidos por no poder satisfacerlos un dia pero se podria despachar de a poco en dias anteriores
# Se asegura que no se sobrepase la cantidad maxima que es posible despachar en un dia.
def constraint_5(model,yva_):
    # Distintos dias en los que se va a estar entregando materiales.
    dias_con_pedidos = yva_['Fe_entrega'].unique()
    dias_con_pedidos = pd.to_datetime(dias_con_pedidos)
    
    # Por cada dia
    for fecha in dias_con_pedidos:
        # Cuento el numero de dias habiles entre la fecha de que se hace la optimizacion (valor obtenido del input)
        # y la fecha considerada en el loop.
        difDias = np.busday_count(fecha_de_hoy.date(), fecha.date())
        
        # Obtengo los pedidos pendientes de entrega para la fecha considerada en el loop.
        yva_fecha = yva_[yva_['Fe_entrega'] == fecha]
        # Y los id's de los pedidos para esa fecha.
        pedidos_unicos = yva_fecha['Doc_venta'].unique()
        
        # La cantidad de material a despachar en todos los pedidos en la fecha considerada en el loop DEBE ser menor o igual
        # a la capacidad de despacho por dia.
        # Edicion Bauti:
        #   /// Pregunta: no habria que generar un rango de fechas para el periodo considerado, asi de esa manera tengo la capacidad de despacho dia a dia.
        #   /// en vez de tener la capacidad de despacho. 
        model.add_constraint(
                (
                model.sum(
                    binarios[nro_pedido] * yva_fecha.loc[yva_fecha['Doc_venta'] == nro_pedido, 'SSU'].sum() for nro_pedido in pedidos_unicos
                    )
                ) 
                <= capacidad_despacho_por_dia * difDias # Aca aumenta la capacidad a despachar dia a dia, pero la cantidad solicitada es siempre la misma (no es que se va considerando lo enviado previamente). 
            )

#%%time
constraint_5(modelo,liberados)
modelo.print_information()

Model: pedidos
 - number of variables: 169
   - binary=8, integer=0, continuous=161
 - number of constraints: 334
   - linear=334
 - parameters: defaults
 - objective: none
 - problem type is: MILP


## FUNCION OBJETIVO

In [91]:
liberados[['Prioridad', 'Doc_venta']].head()

,Prioridad,Doc_venta
0,625.12207,828568192
1,625.12207,828568192
2,632.93457,828568125
3,632.93457,828568125
4,500000.00000,828384334-16446


In [92]:
# Si se decide correr por OTD (On Time Delivery), el modelo va a intentar  despachar la mayor cantidad de pedidos posibles
# teniendo en cuenta que debe sacar los más prioritarios también y utilizando los materiales mas prioritarios

if correr_otd:
    
    modelo.maximize(( 2 * promedio_despacho * modelo.sum(binarios)) 
                    + (modelo.sum(cant_material_a_despachar[k] * yq10_final['PRIORIDAD_FEFO'].iloc[k] for k in cant_material_a_despachar)))
else:
    
    # Si se decide sacar pedidos por prioridad, el modelo intentará sacar los pedidos maás prioritarios con los materiales
    # más prioritarios sin importar si puede sacar más pedidos utilizando otra combinación.
    
    prio_yva = modelo.sum(binarios[i] * liberados['Prioridad'][liberados['Doc_venta'] == i].unique()[0] for i in binarios)
    
    prio_fefo = modelo.sum(cant_material_a_despachar[k] * yq10_final['PRIORIDAD_FEFO'].iloc[k] for k in cant_material_a_despachar)
    
    modelo.maximize(prio_yva + prio_fefo)

In [93]:
modelo.print_information()

Model: pedidos
 - number of variables: 169
   - binary=8, integer=0, continuous=161
 - number of constraints: 334
   - linear=334
 - parameters: defaults
 - objective: maximize
 - problem type is: MILP


In [94]:
solucion = modelo.solve()

In [95]:
print(modelo.get_solve_details())

status  = integer optimal solution
time    = 0.031 s.
problem = MILP
gap     = 0%



In [96]:
modelo.print_solution()

objective: 3146.610
status: OPTIMAL_SOLUTION(2)
  binaria_828419574=1
  binaria_828419583=1
  binaria_828333583=1
  binaria_82838583=1
  binaria_590888255=1
  cantidad_0=5.000
  cantidad_1=2.000
  cantidad_5=2.000
  cantidad_8=2.000
  cantidad_15=25.000
  cantidad_37=2.000
  cantidad_38=1.000
  cantidad_39=1.000
  cantidad_40=2.000


In [97]:
for var in modelo.iter_variables():
    var_value = solucion.get_value(var)
    print('Variable: %s - valor: %s' %(var.get_name(), var_value))

Variable: binaria_828568192 - valor: 0
Variable: binaria_828568125 - valor: 0
Variable: binaria_828384334-16446 - valor: 0
Variable: binaria_828419574 - valor: 1.0
Variable: binaria_828419583 - valor: 1.0
Variable: binaria_828333583 - valor: 1.0
Variable: binaria_82838583 - valor: 1.0
Variable: binaria_590888255 - valor: 1.0
Variable: cantidad_0 - valor: 5.0
Variable: cantidad_1 - valor: 2.0
Variable: cantidad_2 - valor: 0
Variable: cantidad_3 - valor: 0
Variable: cantidad_4 - valor: 0
Variable: cantidad_5 - valor: 2.0
Variable: cantidad_6 - valor: 0
Variable: cantidad_7 - valor: 0
Variable: cantidad_8 - valor: 2.0
Variable: cantidad_9 - valor: 0
Variable: cantidad_10 - valor: 0
Variable: cantidad_11 - valor: 0
Variable: cantidad_12 - valor: 0
Variable: cantidad_13 - valor: 0
Variable: cantidad_14 - valor: 0
Variable: cantidad_15 - valor: 25.0
Variable: cantidad_16 - valor: 0
Variable: cantidad_17 - valor: 0
Variable: cantidad_18 - valor: 0
Variable: cantidad_19 - valor: 0
Variable: ca

In [98]:
solucion_binaria = solucion.get_value_dict(binarios)

despachos_pedidos = pd.DataFrame({'Doc_venta' : solucion_binaria.keys(), 'SE_DESPACHA' : solucion_binaria.values()}).replace([1.0, 0.0], ['SI', 'NO'])
despachos_pedidos

,Doc_venta,SE_DESPACHA
0,828568192,NO
1,828568125,NO
2,828384334-16446,NO
3,828419574,SI
4,828419583,SI
5,828333583,SI
6,82838583,SI
7,590888255,SI


In [99]:
liberados = liberados.merge(despachos_pedidos, how= 'left', on= 'Doc_venta')
liberados

,Doc_venta,ClVt,Solic,Cliente,GClt,Dest_Mcia,Material,SSU,Oficina_de_Venta,Fe_entrega,...,Fecha Recepcion,FechaFact,FePrefEnt_Cab,Status,BqEn,ERROR_FECHA_ENTREGA,Prioridad,Nomb_Dest_Mcia,Cantidad_Total,SE_DESPACHA
0,828568192,ZSCG,3442670,R Y K SRL,EL,1809756,30174172,90.0,N - NEA 3 - I,2023-02-21,...,2022-09-01,2023-01-27,2023-02-15,Liberado,NaN,NO,625.12207,NaN,0.00,NO
1,828568192,ZSCG,3442670,R Y K SRL,EL,1809756,30114422,585.0,N - NEA 3 - I,2023-02-21,...,2022-09-01,2023-01-27,2023-02-15,Liberado,NaN,NO,625.12207,NaN,0.00,NO
2,828568125,ZSCG,1822121,DEKANOR SRL,EF,3395980,12787691,10.0,N - NEA 3 - I,2023-02-21,...,2023-01-01,2023-01-27,2023-02-15,Liberado,NaN,NO,632.93457,NaN,157.00,NO
3,828568125,ZSCG,1822121,DEKANOR SRL,EF,3395980,30174163,90.0,N - NEA 3 - I,2023-02-21,...,2023-01-01,2023-01-27,2023-02-15,Liberado,NaN,NO,632.93457,NaN,9.00,NO
4,828384334-16446,ZCHG,1802978,NaN,EL,1806801,30114751,1.0,N - NOA 3 - I,2023-02-21,...,2022-07-15,2022-06-16,2022-06-16,Liberado,NaN,NO,500000.00000,TUCAGRO SRL,0.00,NO
5,828419574,ZSCG,1801809,NaN,BA,1801809,11789429,4.0,BAU-SE,2023-02-21,...,2022-01-01,2022-06-25,2022-09-15,Liberado,NaN,NO,625.12207,Bauti,6.00,SI
6,828419583,ZSCG,1901809,NaN,BA,1901809,11789429,1.0,BAU-SE,2023-02-21,...,2022-01-01,2022-06-25,2022-09-15,Liberado,NaN,NO,625.12207,Bauti,6.00,SI
7,828333583,ZSCG,1988809,NaN,BA,1988809,12324881,6.0,BAU-SE,2023-02-21,...,2022-01-01,2022-06-25,2022-09-15,Liberado,NaN,NO,625.12207,Bauti,508.00,SI
8,82838583,ZSCG,1982209,NaN,BA,1982209,12950427,5.0,BAU-SE,2023-02-21,...,2022-01-01,2022-06-25,2022-09-15,Liberado,NaN,NO,625.12207,Bauti,5821.00,SI
9,82838583,ZSCG,1982209,NaN,BA,1982209,12950427,1.0,BAU-SE,2023-02-21,...,2022-01-01,2022-06-25,2022-09-15,Liberado,NaN,NO,625.12207,Bauti,5821.00,SI


In [100]:
solucion_cantidad_despachar = solucion.get_value_dict(cant_material_a_despachar)

despachos_cantidad = pd.DataFrame({'Indice_Modelo' : solucion_cantidad_despachar.keys(), 'CANT_A_DESPACHAR' : solucion_cantidad_despachar.values()})
#despachos_cantidad['CANT_A_DESPACHAR'] = despachos_cantidad['CANT_A_DESPACHAR'].astype(int)
despachos_cantidad

,Indice_Modelo,CANT_A_DESPACHAR
0,0,5.0
1,1,2.0
2,2,0.0
3,3,0.0
4,4,0.0
...,...,...
156,156,0.0
157,157,0.0
158,158,0.0
159,159,0.0


In [101]:
yq10_final = yq10_final.merge(despachos_cantidad, how='left', on='Indice_Modelo')
yq10_final['Campaña'].fillna('Embolse', inplace = True)

yq10_final

,HY,MATERIAL,CENTRO,SSU,FECHA_DISPONIBILIDAD,DE,Campaña,Lote,Ubicación,Peso_bolsa,PRIORIDAD_FEFO,Indice_Modelo,CANT_A_DESPACHAR
0,LT626RR2,11789429,6009,6,2023-02-10,LIB3,U,UE9950040N,G12L012C,"17,82",0.5,0,5.0
1,LT722VT3P,12324881,6009,4,2023-02-10,LIB3,W,WE1071181N,G11L136A,"22,79",0.5,1,2.0
2,LT722VT3P,12324881,6009,3,2023-02-10,LIB3,W,WE1071189N,G11L136A,"17,78",0.5,2,0.0
3,LT722VT3P,12324881,6009,1,2023-02-10,LIB3,W,WE1072202N,G11L136A,"18,34",0.5,3,0.0
4,LT722VT3P,12324881,6009,1,2023-02-10,LIB3,W,WE1072220N,G11L136A,"20,42",0.5,4,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,DK72-10RR2,12787701,6244,5,2023-04-11,Emb,Embolse,NaN,NaN,NaN,0.0,156,0.0
157,DK72-10RR2,12787691,6244,57,2023-04-13,Emb,Embolse,NaN,NaN,NaN,0.0,157,0.0
158,DK72-10RR2,12787701,6244,60,2023-04-13,Emb,Embolse,NaN,NaN,NaN,0.0,158,0.0
159,DK72-10RR2,12787691,6244,4,2023-06-25,Emb,Embolse,NaN,NaN,NaN,0.0,159,0.0


## Asignaciones

In [102]:
def clasificar_no_despachados(desp):
    desp['RAZON'] = '-'
    
    desp.loc[desp['SE_DESPACHA'] == 'NO', 'RAZON'] = 0
    
    desp.loc[(desp['RAZON'] == 0) & (desp['SSU'] > desp['Cantidad_Total']), 'RAZON'] = 'Sin stock del material'
    
    for ped in desp['Doc_venta'].unique():
        
        razon_no_despacho = desp.loc[desp['Doc_venta'] == ped, 'RAZON']
        if (razon_no_despacho == 'Sin stock del material').any():
            
            desp.loc[(desp['Doc_venta'] == ped) & (desp['RAZON'] == 0), 'RAZON'] = 'Falta stock de un material en este pedido'
    
    for mat in desp.loc[desp['RAZON'] == 0, 'Material'].unique():
        
        for index , row in desp[(desp['Material'] == mat) & (desp['RAZON'] == 0)].iterrows():
            
            if ((desp.loc[(desp['Material'] == mat) & (desp['SE_DESPACHA'] == 'SI'), 'SSU'].sum() + row['SSU']) > row['Cantidad_Total']):
                
                desp['RAZON'].iloc[index] = 'OTD'
    
    for ped in desp.loc[desp['RAZON'] == 0, 'Doc_venta']:
        
        if any(desp.loc[desp['Doc_venta'] == ped, 'RAZON'] == 'OTD'):
            
            desp.loc[desp['Doc_venta'] == ped, 'RAZON'] = 'OTD'
            
        else:
            
            desp.loc[desp['Doc_venta'] == ped, 'RAZON'] = 'Stock llega mas tarde o falta de capacidad logística'
    
    return desp

In [103]:
yq10_final.reset_index(drop=True, inplace=True)
yq10_final['ID'] = yq10_final.index.values
#yq10_final['ID'] = yq10_final['MATERIAL'].astype(str) + yq10_final['DE'] + yq10_final['Lote'] + yq10_final['Ubicación'].astype(str) + yq10_final['CENTRO'].astype(str)

In [104]:

def asignar_materiales_a_pedidos(liberados_, yq10_):
    
    resultado = pd.DataFrame(
        columns=[
            'Doc_venta', 'Cliente', 'Oficina_de_Venta', 'Fe_entrega', 'Fecha Recepcion',
            'Dest_Mcia', 'Prioridad', 'Status', 'Material', 'ID_Material_Asignado', 'HY',
            'SSU', 'FECHA_DISPONIBILIDAD', 'CENTRO', 'Lote', 'DE', 'Ubicación'
            ]
        )
    
    # Pedidos liberados (YVA), que es posible realizar el envio.
    liberados_si = liberados_[liberados_['SE_DESPACHA'] == 'SI'].copy()
    liberados_si.sort_values(['Fe_entrega','Prioridad'], ascending=[True, False], inplace= True)
    liberados_si.reset_index(drop=True, inplace=True)
    
    # Obtengo los materiales que se van a despachar desde el 6009 (YQ10)
    yq10_si = yq10_[yq10_['CANT_A_DESPACHAR'] > 0].copy()
    yq10_si.sort_values(['FECHA_DISPONIBILIDAD', 'PRIORIDAD_FEFO', 'Peso_bolsa', 'Campaña'], ascending=[True, False, False, True], inplace= True)
    
    # Por cada pedido que es posible responder.
    pedidos_indices = liberados_si.index.to_list()
    for ind in pedidos_indices:
        material = liberados_si.loc[ind, 'Material']
        fecha = liberados_si.loc[ind, 'Fe_entrega']
        # Me quedo con todos los materiales que tengo en stock (YQ10), iguales al solicitado en el pedido, cuya fecha de disponibilidad es menor a la fecha de entrega.
        mat_posibles = yq10_si[(yq10_si['MATERIAL'] == material) & (yq10_si['FECHA_DISPONIBILIDAD'] <= fecha)]
        
        count = 0
        # Esta variable tiene en cuenta la cantidad pedida de material en el envio, vera cuanto de esa cantidad podra responder con el stock que tiene en la YQ10.
        suma = 0
        
        while (suma < liberados_si.loc[ind, 'SSU']) & (count < 10):
            
            mat_posibles = mat_posibles.loc[mat_posibles['CANT_A_DESPACHAR'] > 0, :]
            mat_posibles = mat_posibles.reset_index(drop=True)
            
            if (mat_posibles.loc[0, 'CANT_A_DESPACHAR'] + suma) >= liberados_si.loc[ind, 'SSU']:
                
                add = pd.DataFrame(
                                            {
                                                'Doc_venta': liberados_si.loc[ind, 'Doc_venta'],
                                                'Cliente': liberados_si.loc[ind, 'Nomb_Dest_Mcia'],
                                                'Oficina_de_Venta': liberados_si.loc[ind, 'Oficina_de_Venta'],
                                                'Fe_entrega': liberados_si.loc[ind, 'Fe_entrega'],
                                                'Fecha Recepcion': liberados_si.loc[ind, 'Fecha Recepcion'],
                                                'Dest_Mcia': liberados_si.loc[ind, 'Dest_Mcia'],
                                                'Prioridad': liberados_si.loc[ind, 'Prioridad'],
                                                'Status': liberados_si.loc[ind, 'Status'],
                                                'Material': liberados_si.loc[ind, 'Material'],
                                                'ID_Material_Asignado': mat_posibles.loc[0, 'ID'],
                                                'HY': mat_posibles.loc[0, 'HY'],
                                                'SSU': liberados_si.loc[ind, 'SSU'] - suma,
                                                'FECHA_DISPONIBILIDAD': mat_posibles.loc[0, 'FECHA_DISPONIBILIDAD'],
                                                'CENTRO': mat_posibles.loc[0, 'CENTRO'],
                                                'Lote': mat_posibles.loc[0, 'Lote'],
                                                'DE': mat_posibles.loc[0, 'DE'],
                                                'Ubicación': mat_posibles.loc[0, 'Ubicación']
                                            },
                                        index= [0]
                                        )
                resultado = pd.concat([resultado, add], ignore_index=True)
                
                suma = liberados_si.loc[ind, 'SSU']
                
                yq10_si.loc[yq10_si['ID'] == mat_posibles.loc[0, 'ID'], 'CANT_A_DESPACHAR'] -= liberados_si.loc[ind, 'SSU'] - suma
            
            elif (mat_posibles.loc[0, 'CANT_A_DESPACHAR'] + suma) < liberados_si.loc[ind, 'SSU']:
                
                add = pd.DataFrame(
                                            {   
                                                'Doc_venta': liberados_si.loc[ind, 'Doc_venta'],
                                                'Fe_entrega': liberados_si.loc[ind, 'Fe_entrega'],
                                                'Fecha Recepcion': liberados_si.loc[ind, 'Fecha Recepcion'],
                                                'Dest_Mcia': liberados_si.loc[ind, 'Dest_Mcia'],
                                                'Prioridad': liberados_si.loc[ind, 'Prioridad'],
                                                'Status': liberados_si.loc[ind, 'Status'],
                                                'Material': liberados_si.loc[ind, 'Material'],
                                                'ID_Material_Asignado': mat_posibles.loc[0, 'ID'],
                                                'HY': mat_posibles.loc[0, 'HY'],
                                                'SSU': mat_posibles.loc[0, 'CANT_A_DESPACHAR'],
                                                'FECHA_DISPONIBILIDAD': mat_posibles.loc[0, 'FECHA_DISPONIBILIDAD'],
                                                'CENTRO': mat_posibles.loc[0, 'CENTRO'],
                                                'Lote': mat_posibles.loc[0, 'Lote'],
                                                'DE': mat_posibles.loc[0, 'DE'],
                                                'Ubicación': mat_posibles.loc[0, 'Ubicación']
                                            },
                                        index= [0]
                                        )
                resultado = pd.concat([resultado, add], ignore_index=True)
                
                yq10_si.loc[yq10_si['ID'] == mat_posibles.loc[0, 'ID'], 'CANT_A_DESPACHAR'] -= 0
                
                suma += mat_posibles.loc[0, 'CANT_A_DESPACHAR']
                mat_posibles.loc[0, 'CANT_A_DESPACHAR'] = 0
                count += 1
            
    return resultado

In [105]:
liberados['Doc_venta'] = liberados['Doc_venta'].astype(str)
for indice in liberados.index.to_list():
    liberados.loc[indice, 'Doc_venta'] = liberados.loc[indice, 'Doc_venta'].split('-')[0]

In [106]:
liberados = liberados.reset_index(drop= True)
liberados = clasificar_no_despachados(liberados)
liberados = liberados.drop(columns=['Cantidad_Total'])

In [107]:
asignacion_liberados = asignar_materiales_a_pedidos(liberados, yq10_final)
asignacion_liberados

,Doc_venta,Cliente,Oficina_de_Venta,Fe_entrega,Fecha Recepcion,Dest_Mcia,Prioridad,Status,Material,ID_Material_Asignado,HY,SSU,FECHA_DISPONIBILIDAD,CENTRO,Lote,DE,Ubicación
0,828419574,Bauti,BAU-SE,2023-02-21,2022-01-01,1801809,625.12207,Liberado,11789429,0,LT626RR2,4.0,2023-02-10,6009,UE9950040N,LIB3,G12L012C
1,828419583,Bauti,BAU-SE,2023-02-21,2022-01-01,1901809,625.12207,Liberado,11789429,0,LT626RR2,1.0,2023-02-10,6009,UE9950040N,LIB3,G12L012C
2,828333583,NaN,NaN,2023-02-21,2022-01-01,1988809,625.12207,Liberado,12324881,1,LT722VT3P,2.0,2023-02-10,6009,WE1071181N,LIB3,G11L136A
3,828333583,NaN,NaN,2023-02-21,2022-01-01,1988809,625.12207,Liberado,12324881,8,LT722VT3P,2.0,2023-02-10,6009,WE1072253N,LIB3,G11L136A
4,828333583,Bauti,BAU-SE,2023-02-21,2022-01-01,1988809,625.12207,Liberado,12324881,5,LT722VT3P,2.0,2023-02-10,6009,WE1072225,LIB3,G11L136A
5,82838583,NaN,NaN,2023-02-21,2022-01-01,1982209,625.12207,Liberado,12950427,40,722RR2,2.0,2023-02-10,6009,WE1072821N,LIB3,G13L070B
6,82838583,NaN,NaN,2023-02-21,2022-01-01,1982209,625.12207,Liberado,12950427,37,722RR2,2.0,2023-02-10,6009,UE1150282,LIB3,G12L056D
7,82838583,Bauti,BAU-SE,2023-02-21,2022-01-01,1982209,625.12207,Liberado,12950427,39,722RR2,1.0,2023-02-10,6009,UE1150357,LIB3,G12L062A
8,82838583,Bauti,BAU-SE,2023-02-21,2022-01-01,1982209,625.12207,Liberado,12950427,40,722RR2,1.0,2023-02-10,6009,WE1072821N,LIB3,G13L070B
9,590888255,Bauti,BAU-SE,2023-02-21,2022-01-01,1982209,625.12207,Liberado,12697648,15,DK77-10VT3P,25.0,2023-02-10,6244,WE1170046,LIB3,06007R


In [108]:
liberados['Fe_entrega'] = pd.to_datetime(liberados['Fe_entrega']).dt.date
liberados['Fecha Recepcion'] = pd.to_datetime(liberados['Fecha Recepcion']).dt.date
liberados['FechaFact'] = pd.to_datetime(liberados['FechaFact']).dt.date
liberados['FePrefEnt_Cab'] = pd.to_datetime(liberados['FePrefEnt_Cab']).dt.date

yq10_final['FECHA_DISPONIBILIDAD'] = pd.to_datetime(yq10_final['FECHA_DISPONIBILIDAD']).dt.date

In [109]:
yq10_liberados = yq10_final.copy(deep=True)
yq10_liberados.sort_values('CANT_A_DESPACHAR', ascending= False, inplace= True)
#liberados.sort_values('SE_DESPACHA', ascending= False, inplace= True)
writer = pd.ExcelWriter(nombre_output,engine='xlsxwriter')
inputs.to_excel(writer, sheet_name='INPUTS', index=False)
liberados.to_excel(writer, sheet_name='yva_liberados', index=False)
yq10_liberados.to_excel(writer, sheet_name='yq10_liberados', index=False)
asignacion_liberados.to_excel(writer, sheet_name='pedido_materiales', index=False)
error_fecha.to_excel(writer, sheet_name='error_fecha', index=False)
#writer.save()

In [110]:
yq10_para_bloqueados = yq10_final.copy(deep=True)
yq10_para_bloqueados.reset_index(inplace= True, drop= True)
yq10_para_bloqueados['SSU'] = yq10_para_bloqueados['SSU'] - yq10_para_bloqueados['CANT_A_DESPACHAR']
yq10_para_bloqueados['CANT_A_DESPACHAR'] = 0

In [111]:
indices_yq10_bloq = yq10_para_bloqueados.index.to_list()
yq10_para_bloqueados['Indice_Modelo'] = indices_yq10_bloq
yq10_para_bloqueados['FECHA_DISPONIBILIDAD'] = pd.to_datetime(yq10_para_bloqueados['FECHA_DISPONIBILIDAD'])

In [112]:
bloqueados.reset_index(inplace=True, drop=True)
bloqueados = bloqueados.drop(columns= 'Cantidad_Total')

In [113]:
mat_unique_bloq = list(yq10_para_bloqueados['MATERIAL'].unique())
cantidades_bloq = []
for m in mat_unique_bloq:
    cantidades_bloq.append(yq10_para_bloqueados.loc[yq10_para_bloqueados['MATERIAL'] == m, 'SSU'].sum())

In [114]:
yq10_nueva_bloq = pd.DataFrame({'Material': mat_unique_bloq, 'Cantidad_Total': cantidades_bloq})
yq10_nueva_bloq

,Material,Cantidad_Total
0,11789429,1.0
1,12324881,502.0
2,12697648,902.0
3,12787691,157.0
4,12787701,836.0
5,12950427,5815.0
6,30114414,0.0
7,30114422,0.0
8,30114751,0.0
9,30114758,0.0


In [115]:
bloqueados = bloqueados.merge(yq10_nueva_bloq,how= 'left', on='Material' )
bloqueados['Cantidad_Total'].fillna(0, inplace= True)

In [116]:
modelo_2 = Model(name="bloqueados")

In [117]:
pedidos_bloqueados = bloqueados['Doc_venta'].unique()

In [118]:
binarios = modelo_2.binary_var_dict(pedidos_bloqueados, name = 'bin_bloq')

In [119]:
#cant_material_a_despachar = modelo_2.integer_var_dict(indices_yq10_bloq)
cant_material_a_despachar = modelo_2.continuous_var_dict(indices_yq10_bloq, name = 'cant_bloq')

In [120]:
materiales_pedidos = bloqueados['Material'].unique()

In [121]:
promedio_despacho = bloqueados['SSU'].mean()

In [122]:
#%%time
constraint_1(modelo_2,yq10_para_bloqueados)

In [123]:
#%%time
constraint_2(modelo_2,yq10_para_bloqueados,bloqueados)

In [124]:
#%%time
constraint_3(modelo_2,yq10_para_bloqueados,bloqueados)

In [125]:
#%%time
constraint_4(modelo_2,yq10_para_bloqueados,bloqueados)


In [126]:
#%%time
constraint_5(modelo_2,bloqueados)

In [127]:
if correr_otd:
    modelo_2.maximize((2 * promedio_despacho * modelo_2.sum(binarios)) 
                + (modelo_2.sum(cant_material_a_despachar[k] * yq10_para_bloqueados['PRIORIDAD_FEFO'].iloc[k] for k in cant_material_a_despachar)))
else:
    prio_yva_bloq = modelo_2.sum(binarios[i] * bloqueados['Prioridad'][bloqueados['Doc_venta'] == i].unique()[0] for i in binarios)
    prio_fefo_bloq = modelo_2.sum(cant_material_a_despachar[k] * yq10_para_bloqueados['PRIORIDAD_FEFO'].iloc[k] for k in cant_material_a_despachar)
    modelo_2.maximize(prio_yva_bloq + prio_fefo_bloq)

In [128]:
modelo_2.print_information()

Model: bloqueados
 - number of variables: 170
   - binary=9, integer=0, continuous=161
 - number of constraints: 333
   - linear=333
 - parameters: defaults
 - objective: maximize
 - problem type is: MILP


In [129]:
solucion_bloq = modelo_2.solve()
print(modelo_2.get_solve_details())
modelo_2.print_solution()

status  = integer optimal solution
time    = 0 s.
problem = MILP
gap     = 0%

objective: 1934.150
status: OPTIMAL_SOLUTION(2)
  bin_bloq_828582260=1
  bin_bloq_828561018=1
  cant_bloq_80=1.000
  cant_bloq_89=51.000
  cant_bloq_91=1248.000
  cant_bloq_116=60.000


In [130]:
for var in modelo_2.iter_variables():
    var_value = solucion_bloq.get_value(var)
    print('Variable: %s - valor: %s' %(var.get_name(), var_value))

Variable: bin_bloq_828582260 - valor: 1.0
Variable: bin_bloq_828582148 - valor: 0
Variable: bin_bloq_828580523 - valor: 0
Variable: bin_bloq_828579972 - valor: 0
Variable: bin_bloq_828568136 - valor: 0
Variable: bin_bloq_828561018 - valor: 1.0
Variable: bin_bloq_828560955 - valor: 0
Variable: bin_bloq_828557779 - valor: 0
Variable: bin_bloq_828557761 - valor: 0
Variable: cant_bloq_0 - valor: 0
Variable: cant_bloq_1 - valor: 0
Variable: cant_bloq_2 - valor: 0
Variable: cant_bloq_3 - valor: 0
Variable: cant_bloq_4 - valor: 0
Variable: cant_bloq_5 - valor: 0
Variable: cant_bloq_6 - valor: 0
Variable: cant_bloq_7 - valor: 0
Variable: cant_bloq_8 - valor: 0
Variable: cant_bloq_9 - valor: 0
Variable: cant_bloq_10 - valor: 0
Variable: cant_bloq_11 - valor: 0
Variable: cant_bloq_12 - valor: 0
Variable: cant_bloq_13 - valor: 0
Variable: cant_bloq_14 - valor: 0
Variable: cant_bloq_15 - valor: 0
Variable: cant_bloq_16 - valor: 0
Variable: cant_bloq_17 - valor: 0
Variable: cant_bloq_18 - valor: 0


In [131]:
solucion_binaria_bloqueados = solucion_bloq.get_value_dict(binarios)

despachos_pedidos_bloqueados = pd.DataFrame({'Doc_venta' : solucion_binaria_bloqueados.keys(), 'SE_DESPACHA' : solucion_binaria_bloqueados.values()}).replace([1.0, 0.0], ['SI', 'NO'])
despachos_pedidos_bloqueados

,Doc_venta,SE_DESPACHA
0,828582260,SI
1,828582148,NO
2,828580523,NO
3,828579972,NO
4,828568136,NO
5,828561018,SI
6,828560955,NO
7,828557779,NO
8,828557761,NO


In [132]:
bloqueados = bloqueados.merge(despachos_pedidos_bloqueados, how= 'left', on= 'Doc_venta')

In [133]:
solucion_cantidad_despachar_bloqueados = solucion_bloq.get_value_dict(cant_material_a_despachar)

yq10_para_bloqueados.loc[:, 'CANT_A_DESPACHAR'] = solucion_cantidad_despachar_bloqueados.values()

C:\Users\TALIGENT\AppData\Local\Temp\ipykernel_34684\1207965233.py:3: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  yq10_para_bloqueados.loc[:, 'CANT_A_DESPACHAR'] = solucion_cantidad_despachar_bloqueados.values()


## OUTPUT

In [134]:
yq10_para_bloqueados

,HY,MATERIAL,CENTRO,SSU,FECHA_DISPONIBILIDAD,DE,Campaña,Lote,Ubicación,Peso_bolsa,PRIORIDAD_FEFO,Indice_Modelo,CANT_A_DESPACHAR,ID
0,LT626RR2,11789429,6009,1.0,2023-02-10,LIB3,U,UE9950040N,G12L012C,"17,82",0.5,0,0.0,0
1,LT722VT3P,12324881,6009,2.0,2023-02-10,LIB3,W,WE1071181N,G11L136A,"22,79",0.5,1,0.0,1
2,LT722VT3P,12324881,6009,3.0,2023-02-10,LIB3,W,WE1071189N,G11L136A,"17,78",0.5,2,0.0,2
3,LT722VT3P,12324881,6009,1.0,2023-02-10,LIB3,W,WE1072202N,G11L136A,"18,34",0.5,3,0.0,3
4,LT722VT3P,12324881,6009,1.0,2023-02-10,LIB3,W,WE1072220N,G11L136A,"20,42",0.5,4,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,DK72-10RR2,12787701,6244,5.0,2023-04-11,Emb,Embolse,NaN,NaN,NaN,0.0,156,0.0,156
157,DK72-10RR2,12787691,6244,57.0,2023-04-13,Emb,Embolse,NaN,NaN,NaN,0.0,157,0.0,157
158,DK72-10RR2,12787701,6244,60.0,2023-04-13,Emb,Embolse,NaN,NaN,NaN,0.0,158,0.0,158
159,DK72-10RR2,12787691,6244,4.0,2023-06-25,Emb,Embolse,NaN,NaN,NaN,0.0,159,0.0,159


In [135]:
bloqueados.columns

Index(['Doc_venta', 'ClVt', 'Solic', 'Cliente', 'GClt', 'Dest_Mcia',
       'Material', 'SSU', 'Oficina_de_Venta', 'Fe_entrega',
       'Inicio Siembra Temprana', 'Fin Siembra Temprana',
       'Inicio Siembra Tardia', 'Fin Siembra Tardia', 'Recepcion',
       'Fecha Recepcion', 'FechaFact', 'FePrefEnt_Cab', 'Status', 'BqEn',
       'ERROR_FECHA_ENTREGA', 'Prioridad', 'Nomb_Dest_Mcia', 'Cantidad_Total',
       'SE_DESPACHA'],
      dtype='object')

In [136]:
yq10_para_bloqueados['FECHA_DISPONIBILIDAD'] = pd.to_datetime(yq10_final['FECHA_DISPONIBILIDAD'])
yq10_para_bloqueados.sort_values('CANT_A_DESPACHAR', ascending= False, inplace= True)
bloqueados['Fe_entrega'] = pd.to_datetime(bloqueados['Fe_entrega'])
bloqueados.sort_values('SE_DESPACHA', ascending= False, inplace= True)

In [137]:
bloqueados['FechaFact'] = pd.to_datetime(bloqueados['FechaFact'])
bloqueados['FePrefEnt_Cab'] = pd.to_datetime(bloqueados['FePrefEnt_Cab'])
bloqueados['Fecha Recepcion'] = pd.to_datetime(bloqueados['Fecha Recepcion'])

In [138]:
yq10_para_bloqueados.sort_values('CANT_A_DESPACHAR', ascending= False, inplace= True)
yq10_para_bloqueados['ID'] = yq10_para_bloqueados['MATERIAL'].astype(str) + yq10_para_bloqueados['DE'] + yq10_para_bloqueados['Lote'] + yq10_para_bloqueados['Ubicación'].astype(str) + yq10_para_bloqueados['CENTRO'].astype(str)

In [139]:
bloqueados.sort_values('SE_DESPACHA', ascending= False, inplace= True)

In [140]:
bloqueados = bloqueados.reset_index(drop= True)
bloqueados = clasificar_no_despachados(bloqueados)

In [141]:
bloqueados.loc[bloqueados['SE_DESPACHA'] == 'SI','SSU'].sum()

1360.0

In [142]:
yq10_para_bloqueados['CANT_A_DESPACHAR'].sum()

1360.0

In [143]:
len(yq10_para_bloqueados['ID'].unique())

152

In [144]:
len(yq10_para_bloqueados)

161

In [145]:
asignacion_bloqueados = asignar_materiales_a_pedidos(bloqueados, yq10_para_bloqueados)
asignacion_bloqueados

,Doc_venta,Cliente,Oficina_de_Venta,Fe_entrega,Fecha Recepcion,Dest_Mcia,Prioridad,Status,Material,ID_Material_Asignado,HY,SSU,FECHA_DISPONIBILIDAD,CENTRO,Lote,DE,Ubicación
0,828582260,NaN,NaN,2023-02-21,2023-01-01,3115375,627.075195,Bloq Fac,30114759,30114759LIB3WE1070967G13L088B6009,DK72-20PRO4,1.0,2023-02-10,6009,WE1070967,LIB3,G13L088B
1,828582260,NaN,NaN,2023-02-21,2023-01-01,3115375,627.075195,Bloq Fac,30114759,30114759LIB3WE1072055G13L088C6009,DK72-20PRO4,51.0,2023-02-10,6009,WE1072055,LIB3,G13L088C
2,828582260,NaN,KA6-SE,2023-02-21,2023-01-01,3115375,627.075195,Bloq Fac,30114759,30114759LIB1WE1072080G13L088D6009,DK72-20PRO4,1248.0,2023-02-10,6009,WE1072080,LIB1,G13L088D
3,828561018,NaN,N-M1-E,2023-02-21,2022-05-01,1812785,627.075195,Bloq Fac,30116097,30116097LIB1WE1070670G13L106D6009,DK72-20RR2,60.0,2023-02-10,6009,WE1070670,LIB1,G13L106D


In [146]:
bloqueados.to_excel(writer, sheet_name='yva_bloqueados', index=False)
yq10_para_bloqueados.to_excel(writer, sheet_name='yq10_bloqueados', index=False)
asignacion_bloqueados.to_excel(writer, sheet_name='pedido_materiales_bloq', index=False)
#df_kpi.to_excel(writer, sheet_name='KPIs', index=False)
writer.save()

C:\Users\TALIGENT\AppData\Local\Temp\ipykernel_34684\4008433022.py:5: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [147]:
print('Modelo Finalizado')

Modelo Finalizado


In [148]:
bloqueados_si = bloqueados[bloqueados['SE_DESPACHA'] == 'SI'].copy()
bloqueados_si.sort_values('Prioridad', ascending= False, inplace= True)
yq10_si = yq10_para_bloqueados[yq10_para_bloqueados['CANT_A_DESPACHAR'] > 0].copy()
yq10_si.sort_values(['FECHA_DISPONIBILIDAD', 'PRIORIDAD_FEFO', 'Peso_bolsa', 'Campaña'], ascending=[True, False, False, True], inplace= True)        